### Запуск QtLab

Для начала надо добавить пути со скриптами и драйверами устройств, подключить библиотеки numpy и matplotlib.

In [1]:
from qsweepy import *
from qsweepy.instruments import *
from qsweepy.save_pkl import save_pkl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import interactive
interactive(True)
%matplotlib qt5


Failed loading module Signal_Hound_SA:  name 'signal_hound_dll' is not defined


Чтобы не держать константы далеко в коде, мы записываем сюда параметры кубитов.

In [2]:
#Параметры кубитов.
Qubits = {1: {'Fr': 9.160e9,'F01': 4.149e9, 'I0': 0.,'Ispan': 0.,'P': -30,'Pex': -2,'Span': 200e6},
          2: {'Fr': 9.150e9 ,'F01': 6.74974e9, 'P':-30},
          4: {'Fr': 9.587e9,'F01': 5.91557e9, 'I0': 0.,'Ispan': 0.,'P': -30,'Pex': -2,'Span': 200e6}}
qubit_id = 2

In [2]:
awg = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')

D:\qtlab_replacement\qsweepy\instrument.py 290 ch1_waveform
D:\qtlab_replacement\qsweepy\instrument.py 290 ch2_waveform
D:\qtlab_replacement\qsweepy\instrument.py 290 ch3_waveform
D:\qtlab_replacement\qsweepy\instrument.py 290 ch4_waveform
D:\qtlab_replacement\qsweepy\instrument.py 290 ch1_digital
D:\qtlab_replacement\qsweepy\instrument.py 290 ch2_digital
D:\qtlab_replacement\qsweepy\instrument.py 290 ch3_digital
D:\qtlab_replacement\qsweepy\instrument.py 290 ch4_digital
D:\qtlab_replacement\qsweepy\instrument.py 290 ch5_digital
D:\qtlab_replacement\qsweepy\instrument.py 290 ch6_digital
D:\qtlab_replacement\qsweepy\instrument.py 290 ch7_digital
D:\qtlab_replacement\qsweepy\instrument.py 290 ch8_digital
D:\qtlab_replacement\qsweepy\instrument.py 290 ch1_output
D:\qtlab_replacement\qsweepy\instrument.py 290 ch2_output
D:\qtlab_replacement\qsweepy\instrument.py 290 ch3_output
D:\qtlab_replacement\qsweepy\instrument.py 290 ch4_output
D:\qtlab_replacement\qsweepy\instrument.py 290 trigger_m

In [3]:
awg._visainstrument.ask('TRIG:LEV?')

'1.40000000E+000\n'

In [4]:
awg._visainstrument.ask('AWGC:RMOD?')

'CONT\n'

In [9]:
awg.set_waveform

AttributeError: 'Tektronix_AWG5014' object has no attribute 'set_waveform'

# Спектроскопия

Инстантизируем драйверы устройств для векторного анализатора ('pna') и генератора сигналов ('lo1').

In [4]:
pna = Agilent_N5242A('pna', address = 'TCPIP0::10.20.61.48::inst0::INSTR')
lo1 = Agilent_E8257D('lo1', address = 'TCPIP0::10.20.61.59::inst0::INSTR')

In [5]:
# векторный анализатор в режим линейного свипа
pna.set_sweep_mode("LIN")

False

In [6]:
pna.set_status(1)

True

In [7]:
sweep.sweep(pna, filename='test-pna')

A Jupyter Widget

Started at:  Jul 17 2018 18:20:03
2


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2169: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2170: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2172: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)
root: WARNING  Unsupported type <class 'str'>
root: WARNING  Unsupported type <class 'str'>
root: WARNING  Unsupported type <class 'str'>



Elapsed time: 0 h 0 m 1.12 s


{'S-parameter': (('Frequency',),
  (array([  7.99970000e+09,   7.99970500e+09,   7.99971001e+09, ...,
            8.00968999e+09,   8.00969500e+09,   8.00970000e+09]),),
  array([ 0.84790224+0.0111032j ,  0.84776556+0.0107505j ,
          0.84786314+0.01059819j, ...,  0.70234919+0.40985245j,
          0.70242089+0.40974349j,  0.70287520+0.40882686j]),
  {'log': 20},
  ('Hz',))}

## Спектроскопия через смесители #### ЖМИ МИНЯ


Если подключена импульсная схема, а хочется сделать спектроскопию - то нет проблем! Единственное что нужно сделать, это, во-первых, открыть смесители (подав на них, скажем, 0.3В), и во-вторых, обратить внимание на мощность на выходе. Она будет ниже чем если делать спектроскопию нормально. Ну и шумов тоже получится больше - лучше всего не подавать какие-то сигналы с AWG, а задать оффсеты. Тогда вроде бы шумов меньше получается.

Что тут хорошо: можно переключаться между режимами не выходя из офиса (или из вообще не вставая на кровати). Этот режим очень популярен при измерениях в Черноголовке среди людей, которые не любят в эту самую Черноголовку ездить.
Тем не менее иногда полезно заходить в лабу и посмотреть что же на самом деле там происходит.

In [7]:
# AWG для того чтобы открыть смесители
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')

# Задаём постояннmtые смещения на каналы 1 и 3
awg_tek.set_offset(0.3, channel=1)
awg_tek.set_offset(0.3, channel=3)

# Сами импульсы надо при этом выключить! Будет меньше шуметь (но то не точно).
awg_tek.stop()

root: WARNING  Tektronix_AWG5014 : get all not yet fully functional


## Резонаторная спектроскопия #### НЕЖМИ МИНЯ


In [ ]:
lo1.set_status(0)
pna.set_status(1)
pna.set_power(-50) # 
pna.set_nop(1001)
pna.set_xlim(9.0e9, 10.0e9)
pna.set_bandwidth(100)
freqs = pna.get_freqpoints()
S21 = pna.get_tracedata()

In [ ]:
meas = sweep.sweep(pna, ([0], lambda x: None, 'None'), filename='Single Tone spectroscopy #2')

## Резонатор от мощности (проверка нелинейности резонатора) #### НЕЖМИ МИНЯ


In [ ]:
pna.set_power(-10) # 
pna.set_nop(201) 
#pna.set_xlim(9.8e9, 9.81e9)
pna.set_xlim(9.5e9, 9.7e9)
pna.set_bandwidth(100)
freqs = pna.get_freqpoints()
S21 = pna.get_tracedata()

In [ ]:
resonator_powers = np.arange(16, -51, -6)
meas = sweep.sweep(pna, (resonator_powers, pna.set_power, 'Power, dBm'), filename='Single Tone spectroscopy power dependent #4')

## Фитуем добротность резонатора #### НЕЖМИ МИНЯ


In [ ]:
from resonator_tools import circuit
#plt.plot(freqs, scipy.signal.detrend(np.unwrap(S21[1])))
fitter =circuit.reflection_port(f_data=freqs, z_data_raw=S21[0]*np.exp(1j*S21[1]))
fitter.autofit()
fitter.plotall()

In [ ]:
fitter.fitresults['Qc_err']

In [ ]:
plt.plot(freqs, np.abs(np.gradient(S21[0]*np.exp(1j*S21[1]))))

In [ ]:
powers = np.linspace(-65, 5, 15)
sweep.sweep(pna, (powers, pna.set_power, 'readout power'), filename='Resonator power spectroscopy')

### 1D скан при фиксированной мощности

In [13]:
#### НЕЖМИ МИНЯ

qubit_id = 2
pna.set_nop(1)
#pna.set_xlim(9.58578e9, 9.58578e9)

True

In [22]:
#### НЕЖМИ МИНЯ

#Измерение
lo1.set_status(1)
ex_freqs = np.arange(3.91e9, 3.95e9, 0.2e6)

pna.set_xlim(Qubits[qubit_id]['Fr'], Qubits[qubit_id]['Fr'])
pna.set_nop(1)
pna.set_bandwidth(1)
#pna.set_average_mode('POINT')
#pna.set_average(1)
#pna.set_averages(1)
pna.set_power (-25)
lo1.set_power (10)
#current_src.set_current(-2.5e-3)

header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq', 'Hz'), filename = "ex_{:.1f}dBm #2".format(lo1.get_power()))
except KeyboardInterrupt: pass 

A Jupyter Widget

Started at:  Jul 12 2018 19:02:56
2


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2169: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2170: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2172: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)



Elapsed time: 0 h 3 m 0.82 s


In [ ]:
#### НЕЖМИ МИНЯ

#Измерение
lo1.set_status(1)
ex_freqs = np.arange(5.9e9, 6.0e9, 2e6)

pna.set_xlim(9.45e9, 9.75e9)
pna.set_nop(1001)
pna.set_bandwidth(501)
pna.set_average_mode('POINT')
pna.set_average(1)
pna.set_averages(1)
pna.set_power (-15)
lo1.set_power (13)
#current_src.set_current(-2.5e-3)

header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "ex_{:.1f}dBm #4".format(lo1.get_power()))
except KeyboardInterrupt: pass 

##### 2D скан (мощность второго тона, частота второго тона)

In [18]:
#### НЕЖМИ МИНЯ
Qubit_id = 2
ex_freqs = np.arange(3.7e9, 4.4e9, 1e6)
ex_powers = np.arange(-20, 20, 2)

pna.set_xlim(Qubits[Qubit_id]['Fr'], Qubits[Qubit_id]['Fr'])
pna.set_nop(1)
pna.set_power(Qubits[Qubit_id]['P'])
pna.set_bandwidth(20)
#lo1.set_power(-20)

header = '''#Parameter =
#Ro power = {:f}'''.format( pna.get_power() )

lo1.set_status(1)
#ps.S21vs2DPar(pna, currents, current.set_current, ex_freqs, lo1.set_frequency, filename = "#2{:.1f}dBm".format(lo1.get_power()),\
#           header = header)
ex2D = sweep.sweep(pna, (ex_powers, lo1.set_power, 'Second tone power'), 
                   (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "Two-tone resonator #{} powerscan".format(qubit_id),\
           header = header)
lo1.set_status(0)

A Jupyter Widget

Started at:  Jul 12 2018 18:43:52
2

Elapsed time: 0 h 13 m 55.14 s


True

# Импульсы #### ЖМИ МИНЯ ПОЛНОСТЬЮ

### Создаём девайсы

Инстантизация драйверов устройств.
sa - анализатор спектра,
awg_tek - геннератор сигналов произвольной формы,
adc - аналогово-цифровой преобразователь.

In [7]:
#if not ('pna' in locals()): pna = Agilent_N5242A('pna', address = 'PNA')
#if not ('lo1' in locals()): lo1 = Agilent_E8257D('lo1', address = 'PSG1')
sa = Agilent_N9030A('pxa', address = 'PXA')
#sa = Signal_Hound_SA(name='SA124', serial=61660066)
#lo_ex = Labbrick(name='lo_ex', serial=15249)
lo_ex = lo1
lo_ro = pna
#lo_ro = Labbrick(name='lo_ro', serial=15257)
#awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
adc = Spectrum_M3i2132('adc')


### Загружаем импульсные скрипты всякие

In [8]:
import awg_digital
import awg_iq
import data_reduce
import imp
import fitting
import plotting
import save_pkl
import tomography as tomography_new
import tomography_legacy as tomography
import sweep
imp.reload(awg_digital)
imp.reload(awg_iq)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(save_pkl)
imp.reload(tomography)
imp.reload(tomography_new)
imp.reload(sweep)

<module 'sweep' from 'D:\\qtlab_replacement\\scripts\\sweep.py'>

### Осцилляторы

Генераторы опорных волн для смесителей.
На вход надо подавать около 13 дБ (точка децибельной компрессии).
В качестве источника можно использовать agilent или labbrick. У labbrick фазовые шумы, лучше использвать agilent.
Векторный анализатор можно включить в режим постоянной волны, тогда его тоже можно использовать.

In [9]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм
#lo_ex_pow = 14
#lo_ex_pow = 14
lo_ex_pow = 14
#lo_ex = lo1
lo_ex.set_status(1)
#lo_ex.set_power(lo_ex_pow)
#pna как lo2 (excitation)
#Port1 - выход
#Мощность гетеродина для считывания 10-13 дБм, с учётом разветвителя 13-16 dBm
#lo_ro_pow = 16
lo_ro_pow = 16
lo_ro.set_power(lo_ro_pow)

#lo_ex = pna
pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
lo_ex.set_power(lo_ex_pow)
awg_tek=awg

### AWGшки

In [10]:
# промежуточные частоты для гетеродинной схемы:
ex_if = 50e6
ro_if = 75e6
# клоки генератора и оцифровщика
ex_clock = 1e9
ro_clock = 1e9

rep_rate = 20e3 # частота повторений эксперимента

awg_tek.stop()
awg_tek.set_clock(ex_clock) # клок всех авгшк
awg_tek.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg_tek.run()
# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.

# миксеры подключены к AWG5014 через 6 дБ аттенюаторы. Ставим амплитуду 0.7 В на считывание, 0.4 В на возбуждение.
for channel in range(3,5):
    awg_tek.set_amplitude(0.8, channel=channel)
    awg_tek.set_offset(0.0, channel=channel)
for channel in range(1,3):
    awg_tek.set_amplitude(0.5, channel=channel)
    awg_tek.set_offset(0.0, channel=channel)
# Выходы 1, 2 тектроникс: возбуждение (кубитный тон) через 6 дБ аттенюатор
# Выходы 3, 4 тектроникс: считывание (резонаторный тон) через 6 дБ аттенюатор
iq_ex = awg_iq.awg_iq(awg_tek, awg_tek, 2, 1, lo_ex)
iq_ro = awg_iq.awg_iq(awg_tek, awg_tek, 3, 4, lo_ro)
iq_ex.set_if(ex_if)
iq_ro.set_if(ro_if)
iq_ex.set_sideband_id(-1)
iq_ro.set_sideband_id(-1)
iq_ex.set_frequency(Qubits[qubit_id]['F01'])
iq_ro.set_frequency(Qubits[qubit_id]['Fr'])

# К первому маркере первого канала тектроникса подсоединён триггер оцифровщика.
# Надо подавать какой-нибудь триггер, что-ли.
# А ещё нужно подавать клок. К сожалению наш оцифровщик не может в 1ГГц клок, ему нужен 500МГц клок.
# Будем подавать 500 МГц с цифрового выхода тектроникса.
awg_tek.set_marker1_low(-1, channel=1)
awg_tek.set_marker2_low(-1, channel=1)
awg_tek.set_marker1_high(2, channel=1)
awg_tek.set_marker2_high(2, channel=1)
awg_tek.set_marker1_low(-0.1, channel=2)
awg_tek.set_marker2_low(-0.1, channel=2) 
awg_tek.set_marker1_high(0.1, channel=2) # с клока хватит и такой амплитуды
awg_tek.set_marker2_high(0.1, channel=2) # с клока хватит и такой амплитуды

ro_trg = awg_digital.awg_digital(awg_tek, 1)
osc_trg = awg_digital.awg_digital(awg_tek, 5)
ro_adc_clock = awg_digital.awg_digital(awg_tek, 2) #
osc_adc_clock = awg_digital.awg_digital(awg_tek, 6) #

# тупо клок
ro_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
osc_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
awg_channels = {'iq_ex':iq_ex, 'iq_ro':iq_ro, 'ro_trg':ro_trg, 'osc_trg':osc_trg}
trg_length = 4e-9
pg = tomography_new.pulses(awg_channels)

root: WARNING  Tektronix_AWG5014 : Clock set to 1000000000.0. This is not fully functional yet. To avoid problems, it is better not to change the clock during operation


### Оцифровщик

In [11]:
# настройки оцифровщика
adc.reset()            
adc.set_timeout(10000)
adc.set_clock(ro_clock)

#adc.set_spc_samplerate(smp_rate)
#External trigger
adc.set_trigger_ext0_level0(0) # logical 0 value to 200 mV
adc.set_trigger_ext0_level1(1300) # logical 1 value to 600 mV
adc.trigger_termination_50Ohm()
adc.trigger_mode_pos()
adc.set_trigger_ext0_pulsewidth(0)
adc.disable_trigger_output()
adc.select_channel01()
adc.set_multi_mode()
#adc.set_segmentsize()
adc.set_trigger_ORmask_tmask_ext0()
adc.set_trigger_ANDmask_tmask_ext0()
adc.set_trigger_ORmask_tmask_NO_ch0()
adc.set_trigger_ORmask_tmask_NO_ch1()
adc.set_trigger_ANDmask_tmask_NO_ch0()
adc.set_trigger_ANDmask_tmask_NO_ch1()
# усилитель на оцифровщике
adc.set_input_amp_ch0(50)
adc.set_input_amp_ch1(50)
adc.set_input_offset_ch0(0)
adc.set_input_offset_ch1(0)

#External clock
adc.set_reference_clock(adc.get_clock())

# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
adc_reducer = data_reduce.data_reduce(adc)
adc_reducer.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)
adc_reducer.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.get_if())
adc_reducer.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.get_if())
# Этот измеритель мы как правило используем когда точек не слишком много и все результаты его жизнедеятельности как правило 
# выглядят как ломаные. Чтобы было красиво, давайте лучше сделаем точки (а кривые потом получим фитованые)
adc_reducer.extra_opts['scatter'] = True

### Измерение двухтонового спектра импульсами

In [ ]:
#### НЕЖМИ МИНЯ

#iq_ro.awg_I.__set_dc(1.0)
frequencies = np.arange(5.5e9, 5.9e9, 2e6)
iq_ex.set_if(0.0)
iq_ex._set_dc(0.1)

measurement = sweep.sweep(adc_reducer, (frequencies, iq_ex.set_frequency, 'Second tone frequency'), 
                          filename='Two-tone mixer passthrough')

### Калибровка миксеров

In [12]:
#sa.set_ref(0)
iq_ex.get_calibration(sa)
iq_ro.get_calibration(sa)

d:\qtlab\ReiData\data/calibrations//IQ-if5e+07-rf5.8e+09-sb--1.pkl
d:\qtlab\ReiData\data/calibrations//IQ-if7.5e+07-rf9.6e+09-sb--1.pkl


{'I': (0.34595406626066982+0.48462943803758263j),
 'Q': (0.58268760559944899+0.51625968292930402j),
 'dc': (0.029742644774281197-0.03227259702338911j),
 'num_sidebands': 7,
 'score': -8.6298404224870193}

### Длительность и амплитуда считывающих импульсов

In [14]:
def set_adc_nop(ro_adc_length):
    adc.stop()
    signal_size = int(np.ceil(8./7.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    posttrigger = nop*7/8
    
    print (nop, type(nop))
    adc.set_nop(nop)
    adc.set_post_trigger(posttrigger)

In [ ]:
adc.stop()

In [15]:
 
# ro_amplitude = 0.5 Qubit #2 preliminary
# ro_amplitude = 0.3 Qubit #3 preliminary
ro_dac_amplitude = 0.05
ro_dac_length =0.1e-6 # вообще считывающие импульсы такой длины? тогда что такое adc 
# Ответ: ro_dac_length - это длительность генерируемого прямоугольного импульса
# ro_adc_length - это минимальное время, в течении которого мы считываем этот импульс, когда он прошёл через криостат.
# В криостате импульс деформируется и становится длинее. Сколько времени хотите его интегрировать, столько и ставьте.
# Наверное, имеет смысл сюда поставить на 1-2 времени затухания фотона в резонатор больше, чем ro_dac_length
#ro_adc_length = ro_dac_length*3
#ro_adc_length=0.8e-6 use it for bench in the end
ro_adc_length=0.6e-6  
#adc.set_nums(30000)
#adc.set_software_nums_multi(1)
#set_adc_nop(ro_adc_length)
adc.set_nums(1000)
set_adc_nop(ro_adc_length)
adc.set_nums(100000)
awg_tek.run()

512 <class 'int'>


In [ ]:
#tomography.set_sequence([tomography.ro_rect(ro_amplitude, ro_dac_length, awg_channels)], awg_channels)
plt.figure()
plt.plot(np.mean(np.imag(adc.measure()['Voltage']), axis=0))

In [ ]:
plt.pcolormesh(np.real(adc.measure()['Voltage']));
plt.colorbar()

In [ ]:
adc.stop()

### Прохождение считывающих испульсов разной амплитуды

In [ ]:
def set_pulse_amplitude(x):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    pg.set_seq([pg.p('ro_trg', trg_length, pg.rect, 1), pg.p('iq_ro', ro_dac_length, pg.rect, x)])
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (np.linspace(0, 0.2, 21), set_pulse_amplitude, 'Readout amplitude'), filename='Readout pulse passthrough')

In [ ]:
adc.stop()

### Осцилляции Раби

In [39]:
ex_amplitude = 0.8#смотрим на график в S21-phase ?? как жить что частота от этого зависит
pause_length = 16e-9#пауза между возб.импульсом и считывающим
lengths = np.linspace(0e-6, 100e-9,21)#время наблюдения осцилляций,для фитования частоты достатоно пары периодов
readout_begin = np.max(lengths)
lo1.set_status(True)
def set_ex_length(length):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [pg.p(None, readout_begin-length), 
                pg.p('iq_ex', length, pg.rect, ex_amplitude), 
                pg.p(None, pause_length), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (lengths, set_ex_length, 'Rabi pulse length', 's'), filename='Rabi')
measurement_fitted, fitted_parameters_rabi = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters_rabi['phase'], 
                                                                             fitted_parameters_rabi['freq'], 
                                                                             fitted_parameters_rabi['decay'])
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

A Jupyter Widget

Started at:  Apr 20 2018 17:32:06


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:648: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 1 m 15.55 s


C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [ ]:
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted,annotation=annotation)

In [ ]:
Rabi_fr=fitted_parameters_rabi['freq']
T_pi=0.5/Rabi_fr
T_pi_half=T_pi/2
T_pi,T_pi_half

In [ ]:
measurement

In [ ]:
measurement_fitted

In [ ]:
ro_dac_amplitude

In [ ]:
det=np.sqrt(4*Rabi_fr**2-Qubits[4]['F01']**2)
det

In [ ]:
adc.stop()

### Осцилляции Рамзея

In [ ]:
ex_amplitude = 0.8
# pi2_pulse = 7e-9 #Q2
pi2_pulse = 16e-9 #Q4
#pi2_pulse = T_pi_half #Q4
pause_length = 16e-9
delays = np.linspace(0e-9, 5000e-9, 101)#последовательность задержек между двумя пи/2 импульсами
target_freq_offset = 2e6

readout_begin = np.max(delays)+pi2_pulse*2

def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-pi2_pulse),
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi)),#зачем фаза? 
                pg.p(None, pause_length), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay', 's'), filename='Ramsey')
#measurement = sweep.sweep(adc_sz, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

In [ ]:
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

### Затухание свободной индукции (T1)

In [ ]:
ex_amplitude = 0.2
#pi_pulse = 20e-9
pause_length = 16e-9

In [ ]:
adc.stop()

In [47]:
pi_pulse = 32e-9
delays = np.linspace(0e-6,15e-6, 71)#серия задержек после возбуждения перед считыванием состояния
readout_begin = np.max(delays)+pi_pulse#зачем он тут прибавляет, а потом в коде вычитает? 
# раньше смысл был в том, чтобы менять длительность возбуждающей последовательности, не сдвигая считывание. То есть
# дополнительные импульсы приделываются до считывания (котороене двигается), а не считывание приделывается к концу.
# Сейчас это ещё раз реализовано в коде, которые делает последовательности импульсов (pg.set_seq). Если вдруг у нас будет
# много считываний, то там тоже надо будет мутить какую-нибудь синхронизацию.
def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-pi_pulse),
                pg.p('iq_ex', pi_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'delay'), filename='Decay')

measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_fit)
#save_pkl.save_pkl({'type':'Decay'}, measurement_fitted)
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

A Jupyter Widget

Started at:  Apr 20 2018 17:44:34


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:648: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


KeyboardInterrupt: 

In [ ]:
annotation = 'Decay: {0:4.6g} s'.format(fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Decay'}, measurement_fitted, annotation=annotation)

# Double readout

In [ ]:
adc.set_nums(10000)

sweep_number=5
result=[]
for swp in range(sweep_number):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('ro_trg', trg_length, pg.rect, 1),
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
    measurement=adc.measure()
    #result.append(measurement)
    with open('data.pickle{0}'.format(swp), 'wb') as f:
        pickle.dump(measurement, f)
   # measurement.clear()
    print('{0} done'.format(swp))
    



In [ ]:
#ex_amplitude = 0.2
#pi_pulse = 20e-9
#pause_length = 16e-9
#pi_pulse=20e-9
adc.set_nums(10000)


sweep_number=5
result=[]
for swp in range(sweep_number):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, 0, amp, sigma, alpha),
                pg.p('iq_ex', length, pg.gauss_hd, 0, amp, sigma, alpha),
                pg.p(None,pause_length),
                pg.p('ro_trg', trg_length, pg.rect, 1),
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
    measurement=adc.measure()
   # result.append(measurement)
    with open('data1gaus.pickle{0}'.format(swp), 'wb') as f:
        pickle.dump(measurement, f)
    measurement.clear()
    print('{0} done'.format(swp))
    

In [ ]:
imp.reload(sweep)
imp.reload(save_pkl)
sweep_number=100
adc.set_nums(500)
measurment_data_pre=[]
for swp in range(sweep_number):
    delays = np.linspace(0e-6, 10e-6, 51)
    def set_delay(delay):
        awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
        sequence = [pg.p('ro_trg', trg_length, pg.rect, 1),
                    pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude),
                    pg.p(None, delay),
                    pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)
                   ]
        pg.set_seq(sequence)
        awg_tek.run()
    measurement_pre = sweep.sweep(adc, (delays, set_delay, 'delay'), filename='Decay{0}'.format(swp),time_war_label=False)
    measurement_pre.clear()
   
    
 


In [ ]:
for file_name in range (swp):
    if (file_name==0):
        with open(r'D:\qtlab\ReiData\data\2017-11-03\Voltage Decay{0}.pkl'.format(file_name),'rb') as f:
            data=pkl.load(f,encoding='latin1')
        Voltage_avg=[[np.mean(data[1]['Voltage'][2][i][j]) for i in range(len(data[1]['Voltage'][2][0][1]))]\
                     for j in range(len(data[1]['Voltage'][2][0]))]
        print (Voltage_avg_amp[0])
        print (file_name)
    else:
        with open(r'D:\qtlab\ReiData\data\2017-11-03\Voltage Decay{0}.pkl'.format(file_name),'rb') as f:
            data_r=pkl.load(f,encoding='latin1')
        Voltage_avg=[(np.real(data_r[1]['Voltage'][2][vnum])+np.real(Voltage_avg[vnum]))/2+\
                      (np.imag(data_r[1]['Voltage'][2][vnum])+np.imag(Voltage_avg[vnum]))/2 for vnum \
                      in range(len(data_r[1]['Voltage'][2]))]
        print (data_r[1]['Voltage'][2])
        print (Voltage_avg[0])
        print (file_name)
                      

In [ ]:
sweep_num=80
adc.set_nums(500)
delays = np.linspace(0e-6, 10e-6, 51)
def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, delay)
                pg.p('iq_ex', pi_pulse, pg.rect, ex_amplitude),
                pg.p('ro_trg', trg_length, pg.rect, 1),
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
measurement_pre = sweep.sweep(adc, (delays, set_delay, 'delay'), filename='Decay')
voltage_amp_pre_ONES=[np.sqrt(np.real(measurement_pre['Voltage'][2][1][i])**2+np.imag(measurement_pre['Voltage'][2][1][i])**2) for i in \
             range(len(measurement_pre['Voltage'][2][1]))]
voltage_phase_pre_ONES=[np.arctan(np.real(measurement_pre['Voltage'][2][1][i])/np.imag(measurement_pre['Voltage'][2][1][i])) for i in \
             range(len(measurement_pre['Voltage'][2][1]))] 
plt.plot(voltage_amp_pre_ONES)
plt.show()

In [ ]:
import matplotlib.patches as mpatches

In [ ]:
non_avg_ones=[[voltage_amp_pre_ONES[i][j] for i in range(len(voltage_amp_pre_ONES))]\
              for j in range (len(voltage_amp_pre_ONES[0]))]
avg_ones=[np.mean(ones) for ones in non_avg_ones ]
non_avg_zeros=[[voltage_amp_pre_ZEROS[i][j] for i in range(len(voltage_amp_pre_ZEROS))]\
              for j in range (len(voltage_amp_pre_ZEROS[0]))]
avg_zeros=[np.mean(zeros) for zeros in non_avg_zeros ]
avg_ones_n=[np.mean(ones)-np.mean(avg_ones) for ones in non_avg_ones ]
avg_zeros_n=[np.mean(zeros)-np.mean(avg_zeros) for zeros in non_avg_zeros ]
plt.plot(avg_ones_n)
plt.plot(avg_zeros_n)
blue_patch = mpatches.Patch(color='blue', label='|1>')
red_patch = mpatches.Patch(color='orange', label='|0>')
plt.legend(handles=[blue_patch,red_patch])
plt.show()

In [ ]:
delays = np.linspace(0e-6, 1.5e-6, 51)
def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('ro_trg', trg_length, pg.rect, 1),
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude),
                pg.p(None, delay), 
                pg.p('ro_trg', trg_length, pg.rect, 1),
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude) ]
    pg.set_seq(sequence)
    awg_tek.run()
measurement_post = sweep.sweep(adc_reducer, (delays, set_delay, 'delay'), filename='Decay')

In [ ]:
voltage_amp_post=[np.sqrt(np.real(measurement_post['Voltage'][2][1][i])**2+np.imag(measurement_post['Voltage'][2][1][i])**2) for i in \
             range(len(measurement_post['Voltage'][2][1]))]
voltage_phase_post=[np.arctan(np.real(measurement_post['Voltage'][2][1][i])/np.imag(measurement_post['Voltage'][2][1][i])) for i in \
             range(len(measurement_post['Voltage'][2][1]))]

In [58]:
measurement

{'Mean Voltage (AC)': (('Rabi pulse length', 'Time'),
  (array([  0.00000000e+00,   5.00000000e-09,   1.00000000e-08,
            1.50000000e-08,   2.00000000e-08,   2.50000000e-08,
            3.00000000e-08,   3.50000000e-08,   4.00000000e-08,
            4.50000000e-08,   5.00000000e-08,   5.50000000e-08,
            6.00000000e-08,   6.50000000e-08,   7.00000000e-08,
            7.50000000e-08,   8.00000000e-08,   8.50000000e-08,
            9.00000000e-08,   9.50000000e-08,   1.00000000e-07]),
   array([  0.00000000e+00,   2.00000000e-09,   4.00000000e-09, ...,
            4.09000000e-06,   4.09200000e-06,   4.09400000e-06])),
  array([[-0.00560815-0.00160566j, -0.00740815-0.00180566j,
          -0.00700815-0.00020566j, ...,  0.00469185+0.00689434j,
          -0.00570815+0.00129434j,  0.00559185-0.00610566j],
         [ 0.00694570-0.01156499j,  0.00444570+0.00583501j,
          -0.01035430+0.00333501j, ...,  0.00584570+0.00763501j,
           0.00484570-0.00056499j, -0.00485430+0.

In [59]:
measurement['S21-'][1][0]
S21_pre=measurement['S21-'][1][0][::2]
S21_post=measurement['S21-'][1][0][1::2]
len(measurement['S21-'][1][0])

21

# Optimizing readout pulse settings

In [60]:
#+ 
#amp_x=0.554
#amp_y=0.554
#sigma = 2e-9
#amp_x=0.29
#amp_y=0.29
# Кубит #4, импульсы 8 нс
#pause_length = 2e-9
#length = 10e-9 # Длительность импульса 
#sigma = 2.5e-9 # Срез импульса (в сигмах) = Длительност/сигма
#amp=0.893
#amp_y=0.880
#anharmonicity = -200e6
#alpha_dimensionless = 0.724
# Кубит #4, импульсы 16 нс
#pause_length = 0e-9
#length = 16e-9 # Длительность импульса 
#sigma = 3e-9 # Срез импульса (в сигмах) = Длительност/сигма
#amp=0.322
#anharmonicity = -200e6
#alpha_dimensionless = 0.909#553
#alpha = -alpha_dimensionless/(anharmonicity*2*np.pi)
#xy_acorr = 0.003
#amp_y_raw=0.322#
amp=0.3245
amp2=0.655
length=16e-9
sigma=3e-9
anharmonicity=-227e6
pause_length=0
alpha_dimensionless =1.035
# alpha_dimensionless =1.1
alpha = -alpha_dimensionless/(anharmonicity*2*np.pi)
xy_acorr =0.0
amp_y_raw=0.3245
amp_y=amp_y_raw*np.exp(1j*xy_acorr)

In [61]:
#parametres
#ampx_pi_half
#ampx_pi
#ampy_pi_half
#ampy_pi
ength=16e-9
sigma=3e-9
anharmonicity=-227e6
alpha_dimensionless =1.035
alpha = -alpha_dimensionless/(anharmonicity*2*np.pi)

In [62]:
pi2_pulse= 16e-9
ex_amplitude = 0.8
#ro_dac_amplitude =0.42
#ro_dac_length = 670e-9
pause_length = 0e-9
ro_seq_osc = 10
rel_amp = 0.5
pi2_tukey = 100e-9
amp_tukey = 0.845
pi2_hann = 62e-9
amp_hann = 0.63
pi2_gauss = 32e-9
sigma_gauss=8e-9
amp_gauss = 0.887
anharmonicity=-200e6
alpha_gauss_dimensionless = -0.630
alpha_gauss = alpha_gauss_dimensionless/(2*np.pi*anharmonicity)

ro_dac_amplitude = 0.1
ro_dac_length = 800e-9
ro_seq_osc = 10

length = pi2_gauss
pulse_type = pg.gauss_hd
transition_length = sigma_gauss
amp = amp_gauss
alpha = alpha_gauss

In [63]:
imp.reload(tomography_new)
imp.reload(awg_iq)
imp.reload(data_reduce)
import readout_classifier
imp.reload(readout_classifier)

<module 'readout_classifier' from 'D:\\qtlab_replacement\\scripts\\readout_classifier.py'>

In [64]:
adc.stop()

In [65]:
# To calibrate we need a pi-pulse. Two choices are available, either we use a rectangular pulse or a gaussian pulse.
# The pulse is used to excite the qubit with high probability.
ex_calib_seq = [pg.p('iq_ex', length, pulse_type, amp, transition_length, alpha), pg.p(None, pause_length),\
                pg.p('iq_ex', length, pulse_type, amp, transition_length, alpha), pg.p(None, pause_length)] # gaussian pulse
ex_calib_seq_zero = [pg.p('iq_ex', length, pulse_type,  amp, transition_length, alpha), pg.p(None, pause_length),\
                     pg.p('iq_ex', length, pulse_type, -amp, transition_length, alpha), pg.p(None, pause_length)] # gaussian pulse

#ex_calib_seq = [pg.p('iq_ex', length, pulse_type, amp, transition_length), pg.p(None, pause_length)]*2 # gaussian pulse
#ex_calib_seq_zero = [pg.p('iq_ex', length, pulse_type, amp, transition_length), pg.p(None, pause_length),
#                     pg.p('iq_ex', length, pulse_type, -amp, transition_length), pg.p(None, pause_length),] # gaussian pulse
ro_seq = [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length/3., pg.rect, ro_dac_amplitude), 
                pg.p('iq_ro', ro_dac_length/3., pg.rect, -2*ro_dac_amplitude), 
                pg.p('iq_ro', ro_dac_length/3., pg.rect, 2*ro_dac_amplitude) ]
downsampled_adc = data_reduce.data_reduce(adc)
downsampled_adc.filters['Voltage'] = data_reduce.downsample_reducer(adc, 'Voltage', 1, 62.5e6, iq=True, downsample=8)
adc_sz = tomography_new.sz_measurer(downsampled_adc, ex_calib_seq, ro_seq, pg, ex_seq_zero=ex_calib_seq_zero)
adc_sz.cutoff_start = 0
adc.set_software_nums_multi(1)
adc_sz.repeat_samples = 1
adc_sz.prepare_seqs = [ex_calib_seq_zero, ex_calib_seq]

target_min = lambda: 1-adc_sz.measure()['Calibrated fidelity binary']

class ro_pulse_generator:
    def __init__(self, adc_sz, ro_dac_amplitude, ro_dac_length, ro_seq_osc=1):
        self.ro_dac_amplitude = ro_dac_amplitude
        self.ro_dac_length = ro_dac_length
        self.adc_sz = adc_sz
        self.ro_seq_osc = ro_seq_osc
        self.initial_pulse_rel_amplitude = 0.5
    def set_ro_seq_osc(self, ro_seq_osc):
        self.ro_seq_osc = ro_seq_osc
        self.make_ro_seq()
    def set_initial_pulse_rel_amplitude(self, rel_amp):
        self.initial_pulse_rel_amplitude = rel_amp
        self.make_ro_seq()
    def make_ro_seq(self):
        pl = self.ro_dac_length/(self.ro_seq_osc)
        adc_sz.ro_seq = [pg.p(None, 2e-09),
            pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', pl, pg.rect, self.ro_dac_amplitude*self.initial_pulse_rel_amplitude)]+\
               [pg.p('iq_ro', pl, pg.rect, (-(-1)**i)*self.ro_dac_amplitude) for i in range(self.ro_seq_osc-1)]
    def set_ro_length(self, ro_dac_length):
        self.ro_dac_length = ro_dac_length
        self.make_ro_seq()
    def set_ro_amplitude(self, ro_amplitude):
        self.ro_dac_amplitude = ro_dac_amplitude
        self.make_ro_seq()

ro_dac_lengths = np.linspace(0.0e-6, 1.0e-6, 21)
ro_dac_amplitudes = np.linspace(0.0,0.5, 21)
rel_amps = np.linspace(0.4, 0.6, 21)
#ro_dac_length = 300e-9

ro_pulse_generator_inst = ro_pulse_generator(adc_sz, ro_dac_amplitude, ro_dac_length, 1)
#solution, score = sweep.optimize(target_min, (ro_pulse_generator_inst.set_ro_length, 3e-8), 
#                                             (ro_pulse_generator_inst.set_ro_amplitude, 0.9), maxfun=30)

#ro_dac_length = solution[0]
#ro_dac_amplitude = solution[1]

In [44]:
adc.stop()

In [45]:
ro_pulse_generator_inst.set_ro_amplitude(ro_dac_amplitude)
ro_pulse_generator_inst.set_ro_length(ro_dac_length)
ro_pulse_generator_inst.set_ro_seq_osc(ro_seq_osc)
ro_pulse_generator_inst.set_initial_pulse_rel_amplitude(rel_amp)
ro_amplitude_calib = sweep.sweep(adc_sz,
                                 #(rel_amps, ro_pulse_generator_inst.set_initial_pulse_rel_amplitude, 'initial pulse amplitude'),
                                #(np.arange(1, 10), ro_pulse_generator_inst.set_ro_seq_osc, 'Readout pulse oscillation num'),
                                 (ro_dac_lengths, ro_pulse_generator_inst.set_ro_length, 'Readout pulse length')
                    #(ro_dac_amplitudes, ro_pulse_generator_inst.set_ro_amplitude, 'Readout pulse amplitude')
                                )
#ro_dac_length = ro_amplitude_calib['Calibrated fidelity'][1][0][np.argmax(ro_amplitude_calib['Calibrated fidelity'][2])]

adc.set_software_nums_multi(1)

A Jupyter Widget

Started at:  Apr 20 2018 17:42:19


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:648: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)


ValueError: cannot reshape array of size 20480000 into shape (15000,256,8)

In [ ]:
plt.figure('hists')
plt.plot(adc_sz.predictor_class.proba_points, np.asarray(adc_sz.predictor_class.hists).T)

In [42]:
#setting optimal settings 
print ('Calibrated optimal readout amplitude: ', ro_dac_amplitude, ro_dac_length)
ro_pulse_generator_inst.set_ro_amplitude(ro_dac_amplitude)
ro_pulse_generator_inst.set_ro_length(ro_dac_length)
ro_pulse_generator_inst.set_ro_seq_osc(ro_seq_osc)
calib_meas = adc_sz.measure()
plt.figure('hists')
plt.plot(adc_sz.calib_proba_points, adc_sz.calib_hists.T)
adc_calibrated_reducer = data_reduce.data_reduce(adc_sz.measurer)
#adc_calibrated_reducer = data_reduce.data_reduce(adc)
#adc_calibrated_reducer.filters['sz'] = adc_sz.filter_mean
#adc_calibrated_reducer.filters['sz sh'] = adc_sz.filter
adc_calibrated_reducer.filters['sz'] = adc_sz.filter_binary_mean
print (calib_meas)
adc_calibrated_reducer.measure()

Calibrated optimal readout amplitude:  0.1 8e-07


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:648: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]


ValueError: cannot reshape array of size 20480000 into shape (15000,256,8)

In [43]:
observables = { 'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}
proj_seq = {'Zo': {'pulses':adc_sz.ro_seq, 'operator':observables['Z']} }
reconstruction_basis={'z':{'operator':observables['Z']}}
tomoz = tomography_new.tomography(adc_sz, pg, proj_seq, reconstruction_basis=reconstruction_basis)

AttributeError: 'sz_measurer' object has no attribute 'filter_binary'

In [ ]:
adc_sz.new_calibrate()

In [ ]:
plt.figure()
plt.plot(np.real(adc_sz.calib_feature))
plt.plot(np.imag(adc_sz.calib_feature))

In [ ]:
plt.figure('old avg')
plt.plot(np.real(adc_sz.calib_bg))
plt.plot(np.imag(adc_sz.calib_bg))

In [ ]:
plt.figure('new feature')
plt.plot(np.real(adc_sz.predictor_class.diff))
plt.plot(np.imag(adc_sz.predictor_class.diff))

In [ ]:
adc.stop()

In [66]:
import awg_digital
import awg_iq
import data_reduce
import imp
import fitting
import plotting
import save_pkl
import tomography as tomography_new
import tomography_legacy as tomography
import sweep
imp.reload(awg_digital)
imp.reload(awg_iq)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(save_pkl)
imp.reload(tomography)
imp.reload(tomography_new)
imp.reload(sweep)
pg = tomography_new.pulses(awg_channels)

In [ ]:
adc.stop()

# Rabi

In [69]:
ex_amplitude = 1.0#смотрим на график в S21-phase ?? как жить что частота от этого зависит
pause_length = 16e-9#пауза между возб.импульсом и считывающим
lengths = np.linspace(0e-9, 1e-7,41)#время наблюдения осцилляций,для фитования частоты достатоно пары периодов
tail_length = 12e-9
readout_begin = np.max(lengths)
lo1.set_status(True)
def set_ex_length(length):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [pg.p(None, readout_begin-length), 
                pg.p('iq_ex', length, pg.rect, ex_amplitude), 
                pg.p(None, pause_length)]
    tomoz.set_prepare_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(tomoz, (lengths, set_ex_length, 'Rabi pulse length', 's'), filename='Rabi')
measurement_fitted, fitted_parameters = fitting.fit1d(measurement, fitting.exp_sin_fit, 'z')
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted, annotation=annotation)

NameError: name 'tomoz' is not defined

In [31]:
measurement_fitted, fitted_parameters = fitting.fit1d(measurement, fitting.exp_sin_fit, 'sz')
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted, annotation=annotation)

KeyError: 'sz'

# Ramsey

In [56]:
ex_amplitude = 1.0
# pi2_pulse = 7e-9 #Q2
pi2_pulse = 60e-9 #Q4
#pi2_pulse = T_pi_half #Q4
pause_length = 16e-9
delays = np.linspace(1e-9, 1000e-9, 51)#последовательность задержек между двумя пи/2 импульсами
target_freq_offset = 2e6

readout_begin = np.max(delays)+pi2_pulse*2

def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p(None, readout_begin-pi2_pulse),
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), 
                pg.p(None, delay), 
                pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi)),#зачем фаза? 
                pg.p(None, pause_length), ]
    
    tomoz.set_prepare_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(tomoz, (delays, set_delay, 'Ramsey delay', 's'), filename='Ramsey')
#measurement = sweep.sweep(adc_sz, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.exp_sin_fit(measurement['sz'][1][0], np.reshape(measurement['sz'][2], (1, -1)))
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Ramsey'}, measurement, annotation=annotation)

NameError: name 'tomoz' is not defined

In [ ]:
transition_length
pause_length = 0

In [ ]:
num_pulses = np.arange(0, 45, 1)
y_sign = '-'
y_sign_mul = 1 if y_sign == '-' else -1
pulse_type = pg.gauss_hd
amp = amp_gauss
par1 = 0
length = pi2_gauss
pause_length = 32e-09

def set_ape_length(num_pulses):
#    sequence = [pg.p('iq_ex', length, pulse_type , amp, par1, par2, alpha), pg.p(None, pause_length)]+\
#               [pg.p('iq_ex', length, pulse_type , amp, par1, par2, alpha), pg.p(None, pause_length),
#                pg.p('iq_ex', length, pulse_type , -amp, par1, par2, alpha), pg.p(None, pause_length)]*num_pulses+\
#               [pg.p('iq_ex', length, pulse_type , 1j*y_sign_mul*amp, par1, par2, alpha), pg.p(None, pause_length)]
    sequence = [pg.p('iq_ex', length, pulse_type, amp, transition_length), pg.p(None, pause_length)]+\
               [pg.p('iq_ex', length, pulse_type, amp, transition_length), pg.p(None, pause_length),
                pg.p('iq_ex', length, pulse_type, -amp, transition_length), pg.p(None, pause_length)]*int(np.round(num_pulses))+\
               [pg.p('iq_ex', length, pulse_type, y_sign_mul*1j*amp, transition_length), pg.p(None, 0e-09)]

    tomoz.set_prepare_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(tomoz, (num_pulses, set_ape_length, 'APE identity pulse num'), filename='APE Y'+y_sign)

In [ ]:
num_pulses = 80
y_sign = '+'
y_sign_mul = 1 if y_sign == '+' else -1
#anharmonicity = -223e6
#alphas = np.linspace(-1.0, 1.0, 21)/(200e6*2*np.pi)
alphas = np.linspace(0.75, 1.2, 21)

def set_alpha(alpha_dimensionless):
    alpha = -alpha_dimensionless/(anharmonicity*2*np.pi)
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, alpha), pg.p(None, pause_length)]+\
               [pg.p('iq_ex', length, pg.gauss_hd, amp, alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pg.gauss_hd, -amp, alpha), pg.p(None, pause_length)]*num_pulses+\
               [pg.p('iq_ex', length, pg.gauss_hd, y_sign_mul*amp, alpha), 
                #pg.p(None, pause_length),
                #pg.p('ro_trg', trg_length, pg.rect, 1), 
                #pg.p('iq_ro', ro_dac_length/4., pg.rect, ro_dac_amplitude), 
                #pg.p('iq_ro', ro_dac_length/4., pg.rect, -2*ro_dac_amplitude), 
                #pg.p('iq_ro', ro_dac_length/4., pg.rect, 2*ro_dac_amplitude),
                #pg.p('iq_ro', ro_dac_length/4., pg.rect, -2*ro_dac_amplitude)]
    tomoz.set_prepare_seq(sequence)
    #pg.set_seq(sequence)
    
    awg_tek.run()
#measurement = sweep.sweep(adc_reducer, (alphas, set_alpha, 'HD DRAG alpha'), filename='APE Y'+y_sign+' dependence on HD DRAG alpha')
measurement = sweep.sweep(tomoz, (alphas, set_alpha, 'HD DRAG alpha'), filename='APE Y'+y_sign+' dependence on HD DRAG alpha')

# CPMG

In [68]:
ex_amplitude = 0.79
pi2_pulse = 56e-9
pause_length = 16e-9
N = 1
tays = np.linspace(pi2_pulse*2*N, 10e-6, 21)
tayp = N*pi2_pulse + 2*pi2_pulse
readout_begin = np.max(tays)+pi2_pulse*2
def set_tay(tay):
    sequence = []
    delay = tay/(N*2)-pi2_pulse
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence =  [pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude)]
    sequence += [pg.p(None, delay),
                 pg.p('iq_ex', pi2_pulse*2, pg.rect, ex_amplitude*np.exp(1j*np.pi/2)), 
                 #pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude*np.exp(1j*np.pi/2)),
                 pg.p(None, delay)]*N
    sequence += [pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude),
                 pg.p(None, pause_length), 
                 pg.p('ro_trg', trg_length, pg.rect, 1), 
                 pg.p('iq_ro', ro_dac_length/4., pg.rect, ro_dac_amplitude), 
                 pg.p('iq_ro', ro_dac_length/4., pg.rect, -2*ro_dac_amplitude), 
                 pg.p('iq_ro', ro_dac_length/4., pg.rect, 2*ro_dac_amplitude),
                 pg.p('iq_ro', ro_dac_length/4., pg.rect, -2*ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc_calibrated_reducer,(tays, set_tay, 'CPMG'), filename = 'CPMG N={}'.format(N))

NameError: name 'adc_calibrated_reducer' is not defined

In [ ]:
adc.stop()

### CPMG from thesis

In [ ]:
#ex_amplitude = 0.79
#pi2_pulse = 70e-9
#pause_length = 16e-9
#N = 5
taymax = 10e-06
number_of_pulses = [0, 1, 3, 5, 7, 10, 15, 20, 25, 30]
#number_of_pulses = [0, 1, 3, 5, 7]
#delaymax = taymax - N*2*pi2_pulse
#tays = np.linspace(0, taymax, 21)
#delay = (taymax - N*2*pi_pulse)/(2*N)
#readout_begin = np.max(tays)+pi2_pulse*2
def set_delay(tay):
    sequence = []
    if (N==0):
        delay = tay
    else:
        delay = (tay - N*2*pi2_gauss)/(2*N)
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence =  [pg.p('iq_ex', pi2_gauss, pg.gauss_hd, amp_gauss, sigma_gauss, alpha_gauss)]
    sequence += [pg.p(None, delay),
                 pg.p('iq_ex', pi2_gauss, pg.gauss_hd, amp_gauss*np.exp(1j*np.pi/2), sigma_gauss, alpha_gauss), 
                 pg.p('iq_ex', pi2_gauss, pg.gauss_hd, amp_gauss*np.exp(1j*np.pi/2), sigma_gauss, alpha_gauss), 
                 pg.p(None, delay)]*N
    if (N==0):
        sequence += [pg.p(None, delay)]
    sequence += [pg.p('iq_ex', pi2_gauss, pg.gauss_hd, amp_gauss, sigma_gauss, alpha_gauss),
                 pg.p(None, pause_length)]
                # pg.p('ro_trg', trg_length, pg.rect, 1), 
                # pg.p('iq_ro', ro_dac_length/4., pg.rect, 0.5*ro_dac_amplitude), 
                # pg.p('iq_ro', ro_dac_length/4., pg.rect, -ro_dac_amplitude), 
                # pg.p('iq_ro', ro_dac_length/4., pg.rect, ro_dac_amplitude),
                # pg.p('iq_ro', ro_dac_length/4., pg.rect, -ro_dac_amplitude)]
    #pg.set_seq(sequence)
    tomoz.set_prepare_seq(sequence)
    awg_tek.run()
for i in number_of_pulses:
    N = i
    tays = np.asarray((np.linspace(N*2*pi2_gauss, N*4*pi2_gauss, 11)).tolist() + (np.linspace(N*4*pi2_gauss, taymax/2, 51)).tolist() + (np.linspace(taymax/2, taymax, 31)).tolist())
    measurement = sweep.sweep(tomoz,(tays, set_delay, 'CPMG'), filename = 'CPMG Thesis N={}'.format(N))

In [ ]:
taymax = 10e-06
number_of_pulses = [0, 1, 3, 5, 7, 10, 15, 20, 25, 30]
def set_delay(tay):
    sequence = []
    if (N==0):
        delay = tay
    else:
        delay = tay/(2*N)
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence =  [pg.p('iq_ex', pi2_gauss, pg.gauss_hd, amp_gauss, sigma_gauss, alpha_gauss)]
    sequence += [pg.p(None, delay),
                 pg.p('iq_ex', pi2_gauss, pg.gauss_hd, amp_gauss*np.exp(1j*np.pi/2), sigma_gauss, alpha_gauss), 
                 pg.p('iq_ex', pi2_gauss, pg.gauss_hd, amp_gauss*np.exp(1j*np.pi/2), sigma_gauss, alpha_gauss), 
                 pg.p(None, delay)]*N
    if (N==0):
        sequence += [pg.p(None, delay)]
    sequence += [pg.p('iq_ex', pi2_gauss, pg.gauss_hd, amp_gauss, sigma_gauss, alpha_gauss),
                 pg.p(None, pause_length)]
                # pg.p('ro_trg', trg_length, pg.rect, 1), 
                # pg.p('iq_ro', ro_dac_length/4., pg.rect, 0.5*ro_dac_amplitude), 
                # pg.p('iq_ro', ro_dac_length/4., pg.rect, -ro_dac_amplitude), 
                # pg.p('iq_ro', ro_dac_length/4., pg.rect, ro_dac_amplitude),
                # pg.p('iq_ro', ro_dac_length/4., pg.rect, -ro_dac_amplitude)]
    #pg.set_seq(sequence)
    tomoz.set_prepare_seq(sequence)
    awg_tek.run()
for i in number_of_pulses:
    N = i
    tays = np.asarray((np.linspace(0, N*4*pi2_gauss, 21)).tolist() + (np.linspace(N*4*pi2_gauss, taymax/2, 51)).tolist() + (np.linspace(taymax/2, taymax, 31)).tolist())
    measurement = sweep.sweep(tomoz,(tays, set_delay, 'CPMG'), filename = 'CPMG Thesis N={}'.format(N))

## Double readout 

In [33]:
adc.stop()

In [34]:
delays = np.linspace(0.0e-6,1.5e-6,10)
pause_length = 1e-6
ro_dac_length = 0.8e-6
ro_adc_length = 3.5e-6
adc.set_nums(1000)
set_adc_nop(ro_adc_length)
adc.set_nums(10000)
def set_delays (delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [    pg.p('ro_trg', trg_length, pg.rect, 1), 
                    pg.p('iq_ro', ro_dac_length/4., pg.rect, ro_dac_amplitude), 
                    pg.p('iq_ro', ro_dac_length/4., pg.rect, -2*ro_dac_amplitude), 
                    pg.p('iq_ro', ro_dac_length/4., pg.rect, 2*ro_dac_amplitude),
                    pg.p('iq_ro', ro_dac_length/4., pg.rect, -2*ro_dac_amplitude),
                    pg.p(None, delay),
                    pg.p('iq_ro', ro_dac_length/4., pg.rect, ro_dac_amplitude), 
                    pg.p('iq_ro', ro_dac_length/4., pg.rect, -2*ro_dac_amplitude), 
                    pg.p('iq_ro', ro_dac_length/4., pg.rect, 2*ro_dac_amplitude),
                    pg.p('iq_ro', ro_dac_length/4., pg.rect, -2*ro_dac_amplitude), 
                    pg.p(None, pause_length)]
    pg.set_seq(sequence)
    awg_tek.run()
measurement = sweep.sweep(adc, (delays, set_delays, 'Delay', 's'), filename = r'Double(0)')
    

2048 <class 'int'>


A Jupyter Widget

Started at:  Apr 20 2018 15:49:25


D:\qtlab_replacement\instruments\Tektronix_AWG5014.py:648: ComplexWarning: Casting complex values to real discards the imaginary part
  m1[:] = marker[:len(m1)]



Elapsed time: 0 h 0 m 34.65 s


In [ ]:
from scipy import fft, ifft
plt.plot(measurement['Mean Voltage (AC)'][2][3])


In [ ]:
measurement 

In [ ]:
delays

In [ ]:
import awg_digital
import awg_iq
import data_reduce
import imp
import fitting
import plotting
import save_pkl
import tomography as tomography_new
import tomography_legacy as tomography
import sweep
imp.reload(awg_digital)
imp.reload(awg_iq)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(save_pkl)
imp.reload(tomography)
imp.reload(tomography_new)
imp.reload(sweep)
pg = tomography_new.pulses(awg_channels)

In [ ]:
adc.stop()

In [ ]:
ex_amplitude = 1.0#смотрим на график в S21-phase ?? как жить что частота от этого зависит
pause_length = 16e-9#пауза между возб.импульсом и считывающим
length_tail = 1e-08
lengths = np.linspace(0e-8 + 2*length_tail, 1e-6,61)#время наблюдения осцилляций,для фитования частоты достатоно пары периодов
readout_begin = np.max(lengths)
lo1.set_status(True)
def set_ex_length(length):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [pg.p(None, readout_begin-length), 
                pg.p('iq_ex', length, pg.rect_cos, ex_amplitude, length_tail), 
                pg.p(None, pause_length),
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length/5., pg.rect, 0.5*ro_dac_amplitude), 
                pg.p('iq_ro', ro_dac_length/5., pg.rect, -ro_dac_amplitude), 
                pg.p('iq_ro', ro_dac_length/5., pg.rect, ro_dac_amplitude),
                pg.p('iq_ro', ro_dac_length/5., pg.rect, -ro_dac_amplitude),
                pg.p('iq_ro', ro_dac_length/5., pg.rect, ro_dac_amplitude)]
    awg_tek.run()
measurement = sweep.sweep(adc_calibrated_reducer, (lengths, set_ex_length, 'Rabi pulse length', 's'), filename='Rabi')
measurement_fitted, fitted_parameters = fitting.fit1d(measurement, fitting.exp_sin_fit, 'sz')
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Rabi'}, measurement_fitted, annotation=annotation)

### Rabi amplitude oscillations

In [ ]:
#ex_amps = np.linspace(0.3225,0.3325,21)
#num_pulses = 50
ex_amps = np.linspace(0.88, 0.9, 21)
num_pulses = 256
#pi2_tukey = 88e-9
#length_tail = 12e-9
pause_length = 0e-9
# ro_dac_length=0.2e-6
#readout_begin = np.max((length+pause_length)*num_pulses)
pulse_type = pg.gauss_hd
alpha_tukey = -0.0/(2*np.pi*anharmonicity)
length = pi2_gauss
#sigma = 3e-9
length_transition = sigma_gauss
alpha = 0.
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    #sequence = [pg.p('iq_ex', pi2_tuker, pg.rect, amp), 
    #            pg.p(None, pause_length)]*num_pulses+\
    #            [pg.p('ro_trg', trg_length, pg.rect, 1), 
    #             pg.p('iq_ro', ro_dac_length/5., pg.rect, 0.5*ro_dac_amplitude), 
    #             pg.p('iq_ro', ro_dac_length/5., pg.rect, -ro_dac_amplitude), 
    #             pg.p('iq_ro', ro_dac_length/5., pg.rect, ro_dac_amplitude),
    #             pg.p('iq_ro', ro_dac_length/5., pg.rect, -ro_dac_amplitude),
    #             pg.p('iq_ro', ro_dac_length/5., pg.rect, ro_dac_amplitude)]
    sequence = [pg.p('iq_ex', length, pulse_type, amp, length_transition, alpha), 
                pg.p(None, pause_length)]*num_pulses
    
    tomoz.set_prepare_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(tomoz, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')

In [ ]:
#ex_amps = np.linspace(0.3225,0.3325,21)
#num_pulses = 50
#ex_amps = np.linspace(0.8,1.0,21)
num_pulses = np.arange(0,25)
#pi2_tukey = 88e-9
length_tail = 12e-9
# ro_dac_length=0.2e-6
#readout_begin = np.max((length+pause_length)*num_pulses)
alpha_gauss = -0.0/(2*np.pi*anharmonicity)
pause_length = 0e-9
length = pi2_gauss
def set_num_pulses(num_pulses):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    #sequence = [pg.p('iq_ex', pi2_tuker, pg.rect, amp), 
    #            pg.p(None, pause_length)]*num_pulses+\
    #            [pg.p('ro_trg', trg_length, pg.rect, 1), 
    #             pg.p('iq_ro', ro_dac_length/5., pg.rect, 0.5*ro_dac_amplitude), 
    #             pg.p('iq_ro', ro_dac_length/5., pg.rect, -ro_dac_amplitude), 
    #             pg.p('iq_ro', ro_dac_length/5., pg.rect, ro_dac_amplitude),
    #             pg.p('iq_ro', ro_dac_length/5., pg.rect, -ro_dac_amplitude),
    #             pg.p('iq_ro', ro_dac_length/5., pg.rect, ro_dac_amplitude)]
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp_gauss, alpha_gauss), 
                pg.p(None, pause_length)]*num_pulses
    
    tomoz.set_prepare_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(tomoz, (num_pulses, set_num_pulses, 'Pi half pulse num'), filename='Rabi')

In [ ]:
plt.figure()
plt.plot(np.real(awg_tek.get_waveform(channel=2)))
plt.plot(np.imag(awg_tek.get_waveform(channel=1)))
plt.plot(np.real(awg_tek.get_waveform(channel=3)))
plt.plot(np.imag(awg_tek.get_waveform(channel=4)))

In [ ]:
from scipy.signal import tukey
adc.stop()

In [ ]:
plt.plot(np.real(pg.p('iq_ex', pi2_tuker, pg.rect_cos, 1)))

In [ ]:
#ex_amps = np.linspace(0.3225,0.3325,21)
#num_pulses = 50
ex_amps = np.linspace(0.1,0.7,41)
num_pulses = 4
# ro_dac_length=0.2e-6
#readout_begin = np.max((length+pause_length)*num_pulses)
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), 
                pg.p(None, pause_length)]*num_pulses+\
                [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    
    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')

## Rabi amplitude oscillations pi + pi/2 pulses

In [ ]:
from numpy import pi
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)    
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), 
                pg.p(None, pause_length)]*num_pulses+\
               [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    pg.set_seq(sequence)
    awg_tek.run()
#measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')
#measurement_fitted, fitted_parameters_rabi = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
#def find_ampl(impulse):
    #i=0
    #derivative1, derivative2 =0,0
    #count=0
    #while (0.5*count<impulse/(0.5*pi)):
        #i=i+1;
        #derivative1=derivative2;
        #derivative2=(3*measurement['Mean Voltage (AC)'][i]-4*measurement[i-1]+measurement[i-2])/(2*(0.7/21)
        #if ((derivative1*derivative2)<0) :
            #count=count+1
    #return measurement[i]

#def Rabi_ampl(impulse):

# В таком сиде как оно написано здесь эта процедура не имеет никакого смысла =(
# Если утверждение amp*2=amp_pi верно, то это не нужно. Если нет, то таким обрзом вы это не увидите.

ex_amps = np.linspace(0,0.7,41)
num_pulses=4
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')
measurement_fitted, fitted_parameters_rabi = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
print(fitted_parameters_rabi)
      # return find_ampl(impulse)
#ampl_half_pi=Rabi_ampl(pi/2)   
amp=(3*pi-fitted_parameters_rabi['phase'])/(2*pi*fitted_parameters_rabi['freq'])
amp_pi=(pi*5-fitted_parameters_rabi['phase'])/(2*pi*fitted_parameters_rabi['freq'])
amp, amp_pi

In [ ]:
adc.stop()

### Rabi Y-amplitude oscillations

In [ ]:
ex_amps = np.linspace(0.1,0.7, 101)
num_pulses = 5
#readout_begin = np.max((length+pause_length)*num_pulses)
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, 0, amp, sigma, alpha), 
                pg.p(None, pause_length)]*num_pulses+\
               [pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]
    
    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi Y-pulse amplitude'), filename='Rabi Y-amplitude')

## Amplification of phase errors

In [ ]:
imp.reload(plotting)
adc.stop()

In [ ]:
plt.figure()
plt.plot(np.real(iq_ex.waveform))
plt.plot(np.imag(iq_ex.waveform))

In [ ]:
num_pulses = np.arange(0, 50, 1)
y_sign = '-'
y_sign_mul = 1 if y_sign == '-' else -1

pulse_type = pg.gauss_hd
amp = amp_gauss
length_transition = sigma_gauss # 0
alpha_dimensionless = alpha_gauss_dimensionless
alpha = alpha_dimensionless/(anharmonicity*2*np.pi)
length = pi2_gauss
pause_length = 0e-9

def set_ape_length(num_pulses):
#    sequence = [pg.p('iq_ex', length, pulse_type , amp, par1, par2, alpha), pg.p(None, pause_length)]+\
#               [pg.p('iq_ex', length, pulse_type , amp, par1, par2, alpha), pg.p(None, pause_length),
#                pg.p('iq_ex', length, pulse_type , -amp, par1, par2, alpha), pg.p(None, pause_length)]*num_pulses+\
#               [pg.p('iq_ex', length, pulse_type , 1j*y_sign_mul*amp, par1, par2, alpha), pg.p(None, pause_length)]
    sequence = [pg.p('iq_ex', length, pulse_type, amp, length_transition, alpha), pg.p(None, pause_length)]+\
               [pg.p('iq_ex', length, pulse_type , amp, length_transition,  alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pulse_type , -amp, length_transition,  alpha), pg.p(None, pause_length)]*int(np.round(num_pulses))+\
               [pg.p('iq_ex', length, pulse_type , 1j*y_sign_mul*amp, length_transition, alpha), pg.p(None, pause_length)]

    tomoz.set_prepare_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(tomoz, (num_pulses, set_ape_length, 'APE identity pulse num'), filename='APE Y'+y_sign)

# Scanning over alpha and looking at phase error

In [ ]:
num_pulses = 12
y_sign = '-'
y_sign_mul = 1 if y_sign == '-' else -1
#anharmonicity = -223e6
#alphas = np.linspace(-1.0, 1.0, 21)/(200e6*2*np.pi)
alphas = np.linspace(-2.0, 2.0, 21)
pulse_type = pg.gauss_hd
amp = amp_gauss
length_transition = sigma_gauss # 0
alpha = alpha_dimensionless/(anharmonicity*2*np.pi)
length = pi2_gauss
pause_length = 8e-9

def set_alpha(alpha_dimensionless):
    alpha = alpha_dimensionless/(anharmonicity*2*np.pi)
#    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)]+\
#               [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length),
#                pg.p('iq_ex', length, pg.gauss_hd, -amp, 0, sigma, alpha), pg.p(None, pause_length)]*num_pulses+\
#               [pg.p('iq_ex', length, pg.gauss_hd, 0, y_sign_mul*amp, sigma, alpha), pg.p(None, pause_length)]

    sequence = [pg.p('iq_ex', length, pulse_type, amp, length_transition, alpha), pg.p(None, pause_length)]+\
               [pg.p('iq_ex', length, pulse_type , amp, length_transition,  alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pulse_type , -amp, length_transition,  alpha), pg.p(None, pause_length)]*int(np.round(num_pulses))+\
               [pg.p('iq_ex', length, pulse_type , 1j*y_sign_mul*amp, length_transition, alpha), pg.p(None, pause_length)]

    tomoz.set_prepare_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(tomoz, (alphas, set_alpha, 'HD DRAG alpha'), filename='APE Y'+y_sign+' dependence on HD DRAG alpha')

### X-Y axes angle error

In [ ]:
num_pulses = np.arange(0, 21, 1)
y_sign = '-'
y_sign_mul = 1 if y_sign == '-' else -1

def set_xyea_length(num_pulses):
    sequence = [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)]+\
               [pg.p('iq_ex', length, pg.gauss_hd, 0, amp_y_raw, sigma, alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pg.gauss_hd, 0, amp_y_raw, sigma, alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length),
                pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length),]*(2*num_pulses)+\
               [pg.p('iq_ex', length, pg.gauss_hd, 0, y_sign_mul*amp_y_raw, sigma, alpha), pg.p(None, pause_length),
                pg.p('ro_trg', trg_length, pg.rect, 1), 
                pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]

    pg.set_seq(sequence)
    
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (num_pulses, set_xyea_length, 'X-Y axes angle error pulse num'), filename='XY angle error witness'+y_sign)

### Optimizing X-Y axes angle 

In [ ]:
#angle_corrs = np.linspace(-np.pi/40, np.pi/40, 21)
angle_corrs = np.linspace(-0.03, 0.03, 21)
num_pulses = 20

for signs in ['++++', '+++-', '++-+', '++--', '+-++', '+-+-', '+--+', '+---', 
              '-+++', '-++-', '-+-+', '-+--', '--++', '--+-', '---+', '----']:
    sign_muls = [1 if s == '-' else -1 for s in signs]

    def set_xyea_angle_corr(angle_corr):
        sequence = [pg.p('iq_ex', length, pg.gauss_hd, sign_muls[0]*amp, 0, sigma, alpha), pg.p(None, pause_length)]+\
                   [pg.p('iq_ex', length, pg.gauss_hd, 0, sign_muls[1]*amp_y_raw*np.exp(1j*angle_corr), sigma, alpha), pg.p(None, pause_length),
                    pg.p('iq_ex', length, pg.gauss_hd, 0, sign_muls[1]*amp_y_raw*np.exp(1j*angle_corr), sigma, alpha), pg.p(None, pause_length),
                    pg.p('iq_ex', length, pg.gauss_hd, sign_muls[2]*amp, 0, sigma, alpha), pg.p(None, pause_length),
                    pg.p('iq_ex', length, pg.gauss_hd, sign_muls[2]*amp, 0, sigma, alpha), pg.p(None, pause_length),]*(2*num_pulses)+\
                   [pg.p('iq_ex', length, pg.gauss_hd, 0, sign_muls[3]*amp_y_raw*np.exp(1j*angle_corr), sigma, alpha), pg.p(None, pause_length),
                    pg.p('ro_trg', trg_length, pg.rect, 1), 
                    pg.p('iq_ro', ro_dac_length, pg.rect, ro_dac_amplitude)]

        pg.set_seq(sequence)
        awg_tek.run()
    measurement = sweep.sweep(adc_reducer, (angle_corrs, set_xyea_angle_corr, 'X-Y axes angle detuning (radians)'), filename='XY angle error optimization'+signs)

## Characterizing readout of |0>, |1>, |+>, |->, |i+>, |i-> states

In [ ]:
ex_amplitude, pi2_pulse

In [ ]:
state_prep_seqs = {'|0>': [pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), pg.p(None, pause_length),
                           pg.p('iq_ex', pi2_pulse, pg.rect, -ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, -ex_amplitude), pg.p(None, pause_length)],
                   '|+>': [pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, -ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, 1j*ex_amplitude), pg.p(None, pause_length)],
                   '|1>': [pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, 1j*ex_amplitude), pg.p(None, pause_length),
                           pg.p('iq_ex', pi2_pulse, pg.rect, 1j*ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), pg.p(None, pause_length)],
                   '|->': [pg.p('iq_ex', pi2_pulse, pg.rect, -ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, -ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, 1j*ex_amplitude), pg.p(None, pause_length)],
                   '|i+>': [pg.p('iq_ex', pi2_pulse, pg.rect, 1j*ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, 1j*ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, -1j*ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, -ex_amplitude), pg.p(None, pause_length)],
                   '|i->': [pg.p('iq_ex', pi2_pulse, pg.rect, -1j*ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, -1j*ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, 1j*ex_amplitude), pg.p(None, pause_length),\
                           pg.p('iq_ex', pi2_pulse, pg.rect, ex_amplitude), pg.p(None, pause_length)],}

plt.figure('State readout histograms gaussian')
sz_meas = {}
for state_name, state_prep_seq in state_prep_seqs.items():
    pg.set_seq(state_prep_seq+adc_sz.ro_seq)
    sz_meas[state_name] = np.real(adc_calibrated_reducer.measure()['sz'])
    hist, bin_edges = np.histogram(sz_meas[state_name], bins=adc_sz.calib_proba_points)
    proba_points = (bin_edges[1:]+bin_edges[:-1])/2
    plt.plot(proba_points, hist,label=state_name)
    print ('{0} state sz mean: {1}, std: {2}'.format(state_name, np.mean(sz_meas[state_name]), np.std(sz_meas[state_name])))
    
plt.legend()

In [ ]:
transition_length=sigma_gauss

In [ ]:

state_prep_seqs = {'|0>': [],
                   '|+>': [pg.p('iq_ex', length, pulse_type, amp, transition_length), pg.p(None, pause_length)],
                   '|1>': [pg.p('iq_ex', length, pulse_type, amp, transition_length), pg.p(None, pause_length),\
                           pg.p('iq_ex', length, pulse_type, amp, transition_length), pg.p(None, pause_length)],
                   '|->': [pg.p('iq_ex', length, pulse_type, -amp, transition_length), pg.p(None, pause_length)],
                   '|i+>': [pg.p('iq_ex', length, pulse_type, 1j*amp, transition_length), pg.p(None, pause_length)],
                   '|i->': [pg.p('iq_ex', length, pulse_type, -1j*amp, transition_length), pg.p(None, pause_length)],}

plt.figure('State readout histograms gaussian')
sz_meas = {}
for state_name, state_prep_seq in state_prep_seqs.items():
    pg.set_seq(state_prep_seq+adc_sz.ro_seq)
    sz_meas[state_name] = np.real(adc_sz.measurer.measure()['sz binary'])
    hist, bin_edges = np.histogram(sz_meas[state_name], bins=adc_sz.calib_proba_points)
    proba_points = (bin_edges[1:]+bin_edges[:-1])/2
    plt.plot(proba_points, hist,label=state_name)
    print ('{0} state sz mean: {1}, std: {2}'.format(state_name, np.mean(sz_meas[state_name]), np.std(sz_meas[state_name])))
    
plt.legend()

In [ ]:
# random number generator
n1 = ''.join(['0' if i>0 else '1' for i in sz_meas['|+>'] ])[0::2]
n2 = ''.join(['0' if i>0 else '1' for i in sz_meas['|+>'] ])[1::2]
randoms_raw = ['0' if i>0 else '1' for i in sz_meas['|+>'] ]
randoms = []
for i1,i2 in zip(n1,n2):
    if i1 == '0' and i2 == '1':
        randoms.append('0')
    if i1 == '1' and i2 == '0':
        randoms.append('1')
print ('raw (length {}):'.format(len(randoms_raw)), ''.join(randoms_raw), '\nrandoms (length {}):'.format(len(randoms)), ''.join(randoms))

In [38]:
state_prep_seqs = {'|0>': [],
                   '|+>': [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)],
                   '|1>': [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length),\
                           pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)],
                   '|->': [pg.p('iq_ex', length, pg.gauss_hd, -amp, 0, sigma, alpha), pg.p(None, pause_length)],
                   '|i+>': [pg.p('iq_ex', length, pg.gauss_hd, 0, amp_y, sigma, alpha), pg.p(None, pause_length)],
                   '|i->': [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_y,sigma, alpha), pg.p(None, pause_length)]}

plt.figure('State readout histograms gaussian')
sz_meas = {}
for state_name, state_prep_seq in state_prep_seqs.items():
    pg.set_seq(state_prep_seq+adc_sz.ro_seq)
    sz_meas[state_name] = np.real(adc_calibrated_reducer.measure()['sz'])
    hist, bin_edges = np.histogram(sz_meas[state_name], bins=adc_sz.calib_proba_points)
    proba_points = (bin_edges[1:]+bin_edges[:-1])/2
    plt.plot(proba_points, hist,label=state_name)
    print ('{0} state sz mean: {1}, std: {2}'.format(state_name, np.mean(sz_meas[state_name]), np.std(sz_meas[state_name])))
    
plt.legend()

TypeError: gauss_hd() takes from 5 to 6 positional arguments but 7 were given

In [ ]:
adc.stop()

### Tomography

In [ ]:
#+ 
amp=0.327
amp2=0.657
observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}
proj_seq = {'Xo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, 0, -amp_y, sigma, alpha), pg.p(None, pause_length)]+ro_seq, 
                 'operator': observables['X']},
            'Yo':{'pulses': [pg.p('iq_ex', length, pg.gauss_hd, amp, 0, sigma, alpha), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['Y']},
            '-Xo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, amp_y, sigma, alpha), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-X']},
            '-Yo':{'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp, 0, sigma, alpha), pg.p(None, pause_length)]+ro_seq,
                 'operator': observables['-Y']},
            'Zo': {'pulses':ro_seq, 'operator':observables['Z']} }
reconstruction_basis={'x':{'operator':observables['X']},
                      'y':{'operator':observables['Y']},
                      'z':{'operator':observables['Z']}}
tomo = tomography_new.tomography(adc_sz, pg, proj_seq, reconstruction_basis=reconstruction_basis)

In [ ]:
#+Gh
proj_seq = {'Z': {'pulses':ro_seq, 'operator':observables['Z']}}
reconstruction_basis={'z':{'operator':observables['Z']}}
tomoz = tomography_new.tomography(adc_sz, pg, proj_seq, reconstruction_basis=reconstruction_basis)

In [ ]:
## tomography of |0> state
#adc.set_software_nums_multi(20)
tomo.set_prepare_seq([])
tomo.measure()

In [ ]:
pause_length = 20e-09

# Clifford group

In [ ]:
#+ в оптимальных параметрах амплитуду на 2 умножили и получили пи импульсы))))))000)))
import clifford
imp.reload(clifford)

pulse_type = pg.gauss_hd
amp = amp_gauss
par1 = 0#tail_length # 0
par2 = 0 #sigma
#alpha_dimensionless = 0.0
#alpha = alpha_dimensionless/(anharmonicity*2*np.pi)
length = pi2_gauss

pi2 = {'X/2': {'pulses': [pg.p('iq_ex', length, pulse_type, amp, length_transition, alpha), pg.p(None, pause_length)],
               'unitary': np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]])},
       'Y/2': {'pulses': [pg.p('iq_ex', length, pulse_type, 1j*amp, length_transition, alpha), pg.p(None, pause_length)],
               'unitary': np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]])},
       '-X/2':{'pulses':[pg.p('iq_ex', length, pulse_type, -amp, length_transition, alpha), pg.p(None, pause_length)],
               'unitary': np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]])},
       '-Υ/2':{'pulses':[pg.p('iq_ex', length, pulse_type, -1j*amp, length_transition, alpha), pg.p(None, pause_length)],
               'unitary': np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])},
       'I':   {'pulses':[], 'unitary':np.asarray([[1, 0], [0,1]])}}
#       'X': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, -amp2, par1, alpha), pg.p(None, pause_length)],
#               'unitary': np.asarray([[0, 1],  [1, 0]])},
#       'Y': {'pulses':[pg.p('iq_ex', length, pg.gauss_hd, 0, amp2, sigma, alpha), pg.p(None, pause_length)],
#               'unitary': 1j*np.asarray([[0, -1],  [1, 0]])}}
clifford_group = clifford.generate_group(pi2)#тут меняем наш словарь
print(clifford_group.keys(), len(clifford_group))

In [ ]:
import qutip as qt
qt.sigmay()

# Randomized benchmarking

In [ ]:
#+
import interleaved_benchmarking
imp.reload(interleaved_benchmarking)
imp.reload(sweep)

bench = interleaved_benchmarking.interleaved_benchmarking(tomoz)

#unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
#             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
#             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
#             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}
bench.interleavers = clifford_group
#bench.sequence_length = 10
bench.random_sequence_num = 20
#bench.prepare_random_interleaving_sequences()
#bench.get_points()
#bench.generate_interleaver_sequence_from_names()

In [ ]:
adc.stop()

In [ ]:
#+
#ro_adc_length=0.2e-6 #ВНИМАНИЕ! я это удалю (закомментирую) тут, т.к. оно всё равно здесь ничего не делает, только мозги выносит
seq_lengths = np.arange(0,21, 1)
#seq_lengths = np.arange(0, 100, 10)
random_sequence_ids = np.arange(bench.random_sequence_num)
bench.prepare_random_interleaving_sequences()
clifford_bench = sweep.sweep(bench,
                             (seq_lengths, bench.set_sequence_length_and_regenerate, 'Gate number', ''), 
                             (random_sequence_ids, bench.set_interleaved_sequence, 'Random sequence id', ''))

In [ ]:
#+
imp.reload(fitting)
plt.plot(clifford_bench['distance'][1][0], 
         1-clifford_bench['distance'][2], 
         marker='o', 
         color='black', 
         linestyle='')
#plt.plot(clifford_bench['distance'][1][0], np.mean(1-clifford_bench['distance'][2], axis=1))
clifford_fitresults, clifford_expfit = fitting.exp_fit1d(clifford_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-clifford_bench['distance'][2], axis=1), 
                                                                   (1, len(clifford_bench['distance'][1][0]))))

plt.plot(clifford_bench['distance'][1][0],
         clifford_expfit[0],
         label='Clifford set')
plt.grid()
plt.xlabel('Number of gates')
plt.ylabel('Fidelity')
plt.legend()
print('Clifford fidelity: {0:6.3f}'.format(np.exp(-1/clifford_fitresults[0])))

In [389]:
pi2_gauss

3.2e-08

In [729]:
data

[array([  3.22350000e-01,   3.23750000e-01,   3.19870000e-01,
          3.21480000e-01,   3.22210000e-01,   3.23760000e-01,
          3.23630000e-01,   3.25720000e-01,   3.22880000e-01,
          3.22780000e-01,   3.24500000e-01,   3.21520000e-01,
          2.93690000e-01,   2.68570000e-01,   2.49700000e-01,
          2.32260000e-01,   2.17430000e-01,   1.97500000e-01,
          1.83570000e-01,   1.68940000e-01,   1.55950000e-01,
          1.44690000e-01,   1.33860000e-01,   1.22090000e-01,
          1.16170000e-01,   1.01980000e-01,   9.41900000e-02,
          8.75200000e-02,   7.68300000e-02,   7.52900000e-02,
          6.52000000e-02,   6.26700000e-02,   5.11600000e-02,
          4.71700000e-02,   4.09400000e-02,   3.50000000e-02,
          3.38100000e-02,   2.66900000e-02,   2.62600000e-02,
          2.26400000e-02,   2.24800000e-02,   1.59800000e-02,
          1.02500000e-02,   1.14500000e-02,   6.71000000e-03,
          2.26000000e-03,   4.72000000e-03,   9.70000000e-04,
        

In [487]:
np.asarray(data[1]['z'][1]) - 60*pi2_gauss

array([[  0.00000000e+00,   1.92000000e-07,   3.84000000e-07,
          5.76000000e-07,   7.68000000e-07,   9.60000000e-07,
          1.15200000e-06,   1.34400000e-06,   1.53600000e-06,
          1.72800000e-06,   1.92000000e-06,   1.92000000e-06,
          1.94320000e-06,   1.96640000e-06,   1.98960000e-06,
          2.01280000e-06,   2.03600000e-06,   2.05920000e-06,
          2.08240000e-06,   2.10560000e-06,   2.12880000e-06,
          2.15200000e-06,   2.17520000e-06,   2.19840000e-06,
          2.22160000e-06,   2.24480000e-06,   2.26800000e-06,
          2.29120000e-06,   2.31440000e-06,   2.33760000e-06,
          2.36080000e-06,   2.38400000e-06,   2.40720000e-06,
          2.43040000e-06,   2.45360000e-06,   2.47680000e-06,
          2.50000000e-06,   2.52320000e-06,   2.54640000e-06,
          2.56960000e-06,   2.59280000e-06,   2.61600000e-06,
          2.63920000e-06,   2.66240000e-06,   2.68560000e-06,
          2.70880000e-06,   2.73200000e-06,   2.75520000e-06,
        

In [380]:
time = np.asarray((np.asarray(data2[1]['z'][1][0])-3*2*pi2_gauss).tolist() + (data[1]['z'][1][0]).tolist())
meas = np.asarray(data[1]['z'][2].tolist() + data2[1]['z'][2].tolist())

In [296]:
import fitting_CPMG
from fitting_CPMG import gaussian_CPMG

In [728]:
imp.reload(fitting_CPMG)
from fitting_CPMG import gaussian_CPMG
from fitting_CPMG import fit_CPMG_new
import matplotlib.patches as plm
gamma1 = (10**6)/2.8
gammap = 0
indexes = [0, 1, 5, 10, 20, 30]
adresses = ['19-02-27', '19-18-23', '19-50-37', '20-22-52', '20-55-06', '21-27-03']
time = []
data = []
fitted_data = []
fitted_param = []
p = []
for i in range(6):
    meas = []
    param = []
    with open('D:\qtlab\ReiData\\data\\2018-04-19\\{}-CPMG Thesis N={}\\z CPMG Thesis N={}.pkl'.format(adresses[i], indexes[i], indexes[i]), 'rb') as f:
        time.append(pickle.load(f)[1]['z'][1][0])
    with open('D:\qtlab\ReiData\\data\\2018-04-19\\{}-CPMG Thesis N={}\\z CPMG Thesis N={}.pkl'.format(adresses[i], indexes[i], indexes[i]), 'rb') as f:
        data.append(pickle.load(f)[1]['z'][2])
    if (i==0) or (i==1):
        param, meas = fitting_CPMG.gaussian_CPMG_exp(time[i], data[i])
        #print(i, indexes[i])
        #meas, param = [], []
    else:
        meas, param = fitting_CPMG.fit_CPMG_new(indexes[i], pi2_gauss, gamma1, gammap, tayp, time[i], data[i], gaussian_CPMG)
    meas += 0.5
    fitted_data.append(meas)
    fitted_param.append(param[0])
    p.append(plt.plot(time[i]*1e06, data[i]+0.5, marker = 'o', markersize = 1, linestyle = ''))#, time[i], fitted_data[i])
    #plt.legend('N = {}'.format(indexes[i]))

for i in range(6):
    plt.plot(time[i]*1e06, fitted_data[i], color = '{}'.format(p[i][0].get_color()))
    #p.append(plt.plot(time[i], fitted_data[i]))
#type(p[0][0])
#p[0][0].get_color()
Handels = []
for i in range(6):
    Handels.append(plm.Patch(color = p[i][0].get_color(), label = 'N = {}'.format(indexes[i])))
plt.legend(handles = Handels)

In [731]:
data

(2,
 {'Zo': (('CPMG',),
   (array([  3.20000000e-07,   3.52000000e-07,   3.84000000e-07,
             4.16000000e-07,   4.48000000e-07,   4.80000000e-07,
             5.12000000e-07,   5.44000000e-07,   5.76000000e-07,
             6.08000000e-07,   6.40000000e-07,   6.40000000e-07,
             7.27200000e-07,   8.14400000e-07,   9.01600000e-07,
             9.88800000e-07,   1.07600000e-06,   1.16320000e-06,
             1.25040000e-06,   1.33760000e-06,   1.42480000e-06,
             1.51200000e-06,   1.59920000e-06,   1.68640000e-06,
             1.77360000e-06,   1.86080000e-06,   1.94800000e-06,
             2.03520000e-06,   2.12240000e-06,   2.20960000e-06,
             2.29680000e-06,   2.38400000e-06,   2.47120000e-06,
             2.55840000e-06,   2.64560000e-06,   2.73280000e-06,
             2.82000000e-06,   2.90720000e-06,   2.99440000e-06,
             3.08160000e-06,   3.16880000e-06,   3.25600000e-06,
             3.34320000e-06,   3.43040000e-06,   3.51760000e-06,
 

In [730]:
import pickle
N = 5
with open('D:\qtlab\ReiData\\data\\2018-04-19\\19-50-37-CPMG Thesis N=5\\z CPMG Thesis N=5.pkl', 'rb') as f:
    data = pickle.load(f)
data[1]['z'][1][0] - N*2*pi2_gauss

array([  0.00000000e+00,   3.20000000e-08,   6.40000000e-08,
         9.60000000e-08,   1.28000000e-07,   1.60000000e-07,
         1.92000000e-07,   2.24000000e-07,   2.56000000e-07,
         2.88000000e-07,   3.20000000e-07,   3.20000000e-07,
         4.07200000e-07,   4.94400000e-07,   5.81600000e-07,
         6.68800000e-07,   7.56000000e-07,   8.43200000e-07,
         9.30400000e-07,   1.01760000e-06,   1.10480000e-06,
         1.19200000e-06,   1.27920000e-06,   1.36640000e-06,
         1.45360000e-06,   1.54080000e-06,   1.62800000e-06,
         1.71520000e-06,   1.80240000e-06,   1.88960000e-06,
         1.97680000e-06,   2.06400000e-06,   2.15120000e-06,
         2.23840000e-06,   2.32560000e-06,   2.41280000e-06,
         2.50000000e-06,   2.58720000e-06,   2.67440000e-06,
         2.76160000e-06,   2.84880000e-06,   2.93600000e-06,
         3.02320000e-06,   3.11040000e-06,   3.19760000e-06,
         3.28480000e-06,   3.37200000e-06,   3.45920000e-06,
         3.54640000e-06,

In [570]:
imp.reload(fitting_CPMG)
from fitting_CPMG import gaussian_CPMG
from fitting_CPMG import fit_CPMG_new
gamma1 = (10**6)/2.8
gammap = 0#(10^6)/1.75

In [594]:
time = np.asarray(data[1]['z'][1][0])-N*2*pi2_gauss
measurement_fitted5, fitted_parameters5 = fitting_CPMG.fit_CPMG_new(N, pi2_gauss, gamma1, gammap, tayp, time, data[1]['z'][2], gaussian_CPMG)
time

array([  0.00000000e+00,   3.20000000e-08,   6.40000000e-08,
         9.60000000e-08,   1.28000000e-07,   1.60000000e-07,
         1.92000000e-07,   2.24000000e-07,   2.56000000e-07,
         2.88000000e-07,   3.20000000e-07,   3.20000000e-07,
         4.07200000e-07,   4.94400000e-07,   5.81600000e-07,
         6.68800000e-07,   7.56000000e-07,   8.43200000e-07,
         9.30400000e-07,   1.01760000e-06,   1.10480000e-06,
         1.19200000e-06,   1.27920000e-06,   1.36640000e-06,
         1.45360000e-06,   1.54080000e-06,   1.62800000e-06,
         1.71520000e-06,   1.80240000e-06,   1.88960000e-06,
         1.97680000e-06,   2.06400000e-06,   2.15120000e-06,
         2.23840000e-06,   2.32560000e-06,   2.41280000e-06,
         2.50000000e-06,   2.58720000e-06,   2.67440000e-06,
         2.76160000e-06,   2.84880000e-06,   2.93600000e-06,
         3.02320000e-06,   3.11040000e-06,   3.19760000e-06,
         3.28480000e-06,   3.37200000e-06,   3.45920000e-06,
         3.54640000e-06,

In [573]:
fitted_parameters0, measurement_fitted0 = fitting_CPMG.gaussian_CPMG_exp(time, data[1]['z'][2])

In [596]:
time5 = time

In [575]:
fitted_parameters0,measurement_fitted0

(array([  1.38142807e+06,   3.61027182e-01,  -3.01382164e-02]),
 array([ 0.33088897,  0.32929654,  0.32771114,  0.32613274,  0.32456129,
         0.32299678,  0.32143917,  0.31988842,  0.31834452,  0.31680743,
         0.31527712,  0.31527712,  0.29279462,  0.27177548,  0.25212444,
         0.23375244,  0.21657625,  0.20051803,  0.18550501,  0.17146917,
         0.15834689,  0.14607871,  0.13460906,  0.12388594,  0.11386077,
         0.10448812,  0.09572553,  0.08753327,  0.07987424,  0.07271372,
         0.06601926,  0.05976053,  0.05390918,  0.04843868,  0.04332424,
         0.0385427 ,  0.03407237,  0.02989302,  0.02598569,  0.02233268,
         0.01891744,  0.01572449,  0.01273937,  0.00994854,  0.00733936,
         0.00490001,  0.00261943,  0.00048729, -0.00150607, -0.00336969,
        -0.00511201, -0.00674092, -0.00826381, -0.00968758, -0.01101867,
        -0.01226313, -0.01342659, -0.01451432, -0.01553125, -0.016482  ,
        -0.01737086, -0.01820186, -0.01820186, -0.01949982, 

In [597]:
plt.plot(time5, measurement_fitted5)
plt.plot(time5, data[1]['z'][2], marker = 'o', markersize = 1, linestyle = '')
#plt.legend('N = 10')
print(fitted_parameters5)

[  4.96295516e+05   2.85688543e-01  -6.04324674e-03]


In [581]:
print(fitted_parameters10)

[  4.05634810e+05   2.71951386e-01  -1.27497213e-02]


In [ ]:
clifford_location='D:\\qtlab\\ReiData\\data\\2017-11-02\\21-07-16'
name='distance None'
clifford_bench = {}
clifford_bench = save_pkl.load_pkl(name, clifford_location)[1]
#clifford_bench['distance']
clifford_bench['Pulse sequence']

In [ ]:
print([clifford_bench['Pulse sequence'][2][1][i][::2] for i in range(len(clifford_bench['Pulse sequence'][2][1]))])
print([clifford_bench['Pulse sequence'][2][1][i][::2] for i in range(len(clifford_bench['Pulse sequence'][2][1]))])

In [ ]:
clifford_bench['distance'][2][1,:]

In [ ]:
not_location='D:\\qtlab\\ReiData\\data\\2017-11-02\\22-14-16'
name='distance None'
not_bench = {}
not_bench = save_pkl.load_pkl(name, not_location)[1]
#clifford_bench['distance']
pass

In [ ]:
hadamard_location='D:\\qtlab\\ReiData\\data\\2017-11-02\\21-40-30'
name='distance None'
hadamard_bench = {}
hadamard_bench = save_pkl.load_pkl(name, hadamard_location)[1]
#clifford_bench['distance']
pass

## interleaved benchmarking of H gate and NOT

In [ ]:
import interleaved_benchmarking
imp.reload(interleaved_benchmarking)

bench = interleaved_benchmarking.interleaved_benchmarking(tomoz)

bench.interleavers = clifford_group
bench.random_sequence_num = 10
bench.target_gate = clifford_group['Y/2 X/2 X/2']['pulses'] # NOT
bench.target_gate_name = 'Hadamard'
bench.target_gate_unitary = clifford_group['Y/2 X/2 X/2']['unitary']


In [ ]:
seq_lengths = np.arange(0, 51, 5)
random_sequence_ids = np.arange(bench.random_sequence_num)
bench.prepare_random_interleaving_sequences()
hadamard_bench = sweep.sweep(bench,
                             (seq_lengths, bench.set_sequence_length_and_regenerate, 'Gate number', ''), 
                             (random_sequence_ids, bench.set_interleaved_sequence, 'Random sequence id', ''))

In [ ]:
imp.reload(fitting)
plt.plot(clifford_bench['distance'][1][0], 1-clifford_bench['distance'][2], marker='o', color='black', linestyle='')
#plt.plot(clifford_bench['distance'][1][0], np.mean(1-clifford_bench['distance'][2], axis=1), label='Cliffords (average)')
plt.plot(hadamard_bench['distance'][1][0], 1-hadamard_bench['distance'][2], marker='o', color='blue', linestyle='')
#plt.plot(hadamard_bench['distance'][1][0], np.mean(1-hadamard_bench['distance'][2], axis=1), label='Hadamard interleaved (average)')
clifford_fitresults, clifford_expfit = fitting.exp_fit1d(clifford_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-clifford_bench['distance'][2], axis=1), 
                                                                   (1, len(clifford_bench['distance'][1][0]))))
hadamard_fitresults, hadamard_expfit = fitting.exp_fit1d(hadamard_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-hadamard_bench['distance'][2], axis=1), 
                                                                   (1, len(hadamard_bench['distance'][1][0]))))
plt.plot(clifford_bench['distance'][1][0],clifford_expfit[0], label='Cliffords (fit)')
plt.plot(hadamard_bench['distance'][1][0],hadamard_expfit[0], label='Hadamard (fit)')#, linestyle='-', marker='')
print('''Clifford fidelity: {0:6.3f}
Hadamard+Clifford fidelity: {1:6.3f}
Hadamard error rate: {2:6.3f}'''.format(np.exp(-1/clifford_fitresults[0]), 
                                        np.exp(-1/hadamard_fitresults[0]),
                                       1-np.exp(-1/hadamard_fitresults[0])/np.exp(-1/clifford_fitresults[0])))
plt.xlabel(clifford_bench['distance'][0][0])
plt.ylabel('Fidelity')
plt.legend()
plt.grid()

In [ ]:
import interleaved_benchmarking
imp.reload(interleaved_benchmarking)

bench = interleaved_benchmarking.interleaved_benchmarking(tomoz)

bench.interleavers = clifford_group
bench.random_sequence_num = 10
bench.target_gate = clifford_group['X/2 X/2']['pulses'] # NOT
bench.target_gate_name = 'NOT'
bench.target_gate_unitary = clifford_group['X/2 X/2']['unitary']


In [ ]:
seq_lengths = np.arange(0, 51, 5)
random_sequence_ids = np.arange(bench.random_sequence_num)
bench.prepare_random_interleaving_sequences()
not_bench = sweep.sweep(bench,
                             (seq_lengths, bench.set_sequence_length_and_regenerate, 'Gate number', ''), 
                             (random_sequence_ids, bench.set_interleaved_sequence, 'Random sequence id', ''))

In [ ]:
imp.reload(fitting)
plt.plot(clifford_bench['distance'][1][0], 1-clifford_bench['distance'][2], marker='o', color='black', linestyle='')
#plt.plot(clifford_bench['distance'][1][0], np.mean(1-clifford_bench['distance'][2], axis=1), label='Cliffords (mean)')
plt.plot(hadamard_bench['distance'][1][0], 1-hadamard_bench['distance'][2], marker='o', color='blue', linestyle='')
#plt.plot(hadamard_bench['distance'][1][0], np.mean(1-hadamard_bench['distance'][2], axis=1), label='H interleaved (mean)')
plt.plot(not_bench['distance'][1][0], 1-not_bench['distance'][2], marker='o', color='red', linestyle='')
#plt.plot(not_bench['distance'][1][0], np.mean(1-not_bench['distance'][2], axis=1), label='NOT interleaved (mean)')
clifford_fitresults, clifford_expfit = fitting.exp_fit1d(clifford_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-clifford_bench['distance'][2], axis=1), 
                                                                   (1, len(clifford_bench['distance'][1][0]))))
hadamard_fitresults, hadamard_expfit = fitting.exp_fit1d(hadamard_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-hadamard_bench['distance'][2], axis=1), 
                                                                   (1, len(hadamard_bench['distance'][1][0]))))
not_fitresults, not_expfit = fitting.exp_fit1d(not_bench['distance'][1][0], 
                                                        np.reshape(np.mean(1-not_bench['distance'][2], axis=1), 
                                                                   (1, len(not_bench['distance'][1][0]))))
plt.plot(clifford_bench['distance'][1][0],clifford_expfit[0], label='Cliffords (fit)')
plt.plot(hadamard_bench['distance'][1][0],hadamard_expfit[0], label='Hadamard (fit)')#, linestyle='-', marker='')
plt.plot(not_bench['distance'][1][0],not_expfit[0], label='NOT (fit)')
print('''Clifford fidelity: {0:6.3f}
Hadamard+Clifford fidelity: {1:6.3f}
NOT+Clifford fidelity: {3:6.3f}
Hadamard error rate: {2:6.3f}
NOT error rate: {4:6.3f}'''.format(np.exp(-1/clifford_fitresults[0]), 
                                        np.exp(-1/hadamard_fitresults[0]),
                                       1-np.exp(-1/hadamard_fitresults[0])/np.exp(-1/clifford_fitresults[0]), 
                                        np.exp(-1/not_fitresults[0]),
                                       1-np.exp(-1/not_fitresults[0])/np.exp(-1/clifford_fitresults[0])))
plt.xlabel(clifford_bench['distance'][0][0])
plt.ylabel('Fidelity')
plt.legend()
plt.grid()

In [ ]:
bench.target_gate = bench.interleavers['X']['pulses']
bench.target_gate_unitary = bench.interleavers['X']['unitary']
bench.target_gate_name = 'X (benchmarking)'
tfunc = lambda : np.mean(bench.measure()['Euclidean distance'])
class hd_pulse_generator:
    def __init__(self, pulse_setter, pg, channel):
        self.amp_x = -amp_x_hd
        self.amp_y = amp_x_hd
        self.sigma = sigma_hd
        self.length = length_hd
        self.pause_length = pause_length_hd
        self.alpha = alpha_hd
        
        self.pg = pg
        self.channel = channel
        self.pulse = self.pulse_gen()
        self.pulse_setter = pulse_setter
    def set_real_amplitude(self,x):
        self.amp_x = x
        self.pulse_setter(self.pulse_gen())
    def set_imag_amplitude(self,y):
        self.amp_y = y
        self.pulse_setter(self.pulse_gen())
    def set_sigma(self,s):
        self.sigma = s
        self.pulse_setter(self.pulse_gen())
    def set_length(self,l):
        self.length = l
        self.pulse_setter(self.pulse_gen())
    def set_pause_length(self, p):
        self.pause_length = p
        self.pulse_setter(self.pulse_gen())
    def set_alpha(self, a):
        self.alpha = a
        self.pulse_setter(self.pulse_gen())
    def pulse_gen(self):
        return [pg.p(self.channel, self.length, self.pg.gauss_hd, self.amp_x+1j*self.amp_y, 0, self.sigma, self.alpha), pg.p(None, self.pause_length)]
    
xg = hd_pulse_generator(bench.set_target_pulse, pg, 'iq_ex')

In [ ]:
sol, score = sweep.optimize(tfunc,  (xg.set_real_amplitude, xg.amp_x), 
                                    (xg.set_imag_amplitude, xg.amp_y),
                                    (xg.set_alpha, xg.alpha))

In [ ]:
amps = np.linspace(0.925, 0.975, 21)
alphas = np.linspace(-2e-9, 2e-9, 11)
sweep.sweep(bench, (amps, xg.set_real_amplitude, 'Amplitude'), (alphas, xg.set_alpha, 'HD DRAG coeff'), filename='Randomized interleaved benchmarging X-rotation')

In [ ]:
data = save_pkl.load_pkl('Euclidean distance Randomized interleaved benchmarging X-rotation', 'D:\\qtlab\\ReiData\\data\\2017-07-02\\23-03-34')
plt.figure()
plt.pcolormesh(data[1]['Euclidean distance'][1][1], data[1]['Euclidean distance'][1][0], data[1]['Euclidean distance'][2][:,:,5])
plt.colorbar()

In [ ]:
data[1]

In [ ]:
data[1]['Euclidean distance']

In [ ]:
meas = bench.measure()

In [ ]:
meas['Xo']

In [ ]:
import plotting
imp.reload(plotting)
plotting.plot_measurement(bench.reference_benchmark_result)

# Optimizing Drag

In [ ]:
num_interleved_pulses = 2
alphas = np.linspace(-1e-8, 1e-8, 201)
def set_drag (alpha):
    ramsey_half = [pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                 pg.p(None, pause_length_hd)]
    interleaved_identity = [pg.p('iq_ex', length_hd, pg.gauss_hd, -amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd),
                           pg.p('iq_ex', length_hd, pg.gauss_hd, amp_x_hd, 0, sigma_hd, alpha), 
                           pg.p(None, pause_length_hd)]
    tomo.set_prepare_seq(ramsey_half+interleaved_identity*num_interleved_pulses+ramsey_half)
    
measurement = sweep.sweep(tomo, (alphas, set_drag, 'delta_freq'), filename='DRAG', output=False)

### Randomized Benchmarking

In [ ]:
pause_pulse = tomography.pause(pause_length, awg_channels)
pulses = {0:tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          1:tomography.ex_gauss_hd(amp_y_hd, -amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          2:tomography.ex_gauss_hd(-amp_x_hd, -amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          3:tomography.ex_gauss_hd(-amp_y_hd, amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi)}

unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}

observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}

initial = np.asarray([0, 1]).T
random_pulse_ids = np.arange(20)
num_gates = np.linspace(0, 100, 11)

sequences = []
pulse_sequences = []
theor_projs = {o:np.zeros((len(num_gates), len(random_pulse_ids))) for o in observables.keys()}
rho_t = np.zeros((len(num_gates), len(random_pulse_ids),2,2), dtype=np.complex)
psis = np.zeros((len(num_gates), len(random_pulse_ids),2), dtype=np.complex)
n=0

def set_num_gates(n_new):
    global n
    n = n_new

def set_random_sequence(id):
    sequence = np.random.randint (4, size = n).tolist()
    sequence_pulses = [j for i in [[pulses[i], pause_pulse] for i in sequence] for j in i]
    sequence_unitaries = [unitaries[i] for i in sequence]
    psi = initial.copy()
    
    for U in sequence_unitaries:
        psi = np.dot(U, psi)
    
    rho_t[n,id,:,:] = np.einsum('i,j->ij', np.conj(psi), psi)
    psis[n,id,:] = psi
    
    sequences.append(sequence)
    for o in observables.keys():
        theor_projs[o][n,id] = np.dot(np.dot(np.conj(psi), observables[o]), psi)
    
    tomo_hd.set_prepare_seq(sequence_pulses)
    pulse_sequences.append(sequence_pulses)
    
m = sweep.sweep(tomo_hd, (num_gates, set_num_gates, 'gate num'), (random_pulse_ids, set_random_sequence, 'random seq id'), filename='RB', output=False)
X = (m['X'][2]-m['-X'][2])/2.
Y = (m['Y'][2]-m['-Y'][2])/2.
Z = m['Z'][2]
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('nmi,nmji,nmj->nm', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

In [ ]:
plt.plot(m['Fidelity'][1]m['Fidelity'])

In [ ]:
rho_m

In [ ]:
plt.plot(m['Fidelity'][1][0], np.mean(m['Fidelity'][2],axis=1))
plt.xlabel('gate num')
plt.ylabel('fidelity')
plt.grid()

In [ ]:
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('mni,mnji,mnj->mn', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

In [ ]:
m

In [ ]:
fidelities

In [ ]:
psis

In [ ]:
rho_m

In [ ]:
rho_t

In [ ]:
measurement['theory'] = thoer_proj
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, measurement)

In [ ]:
print(theor_projs)
print(measurement)
print(psis)
print (sequences)
print (rho_t)
print (rho_m)

In [ ]:
np.dot(np.conj(psis[1]), np.dot(observables['Y'],psis[1]))

In [ ]:
num_pulses = np.round(np.arange(0, 41)).astype(np.int)

def set_num_pulses(n):
    tomoz.set_prepare_seq([tomography.ex_gauss(amp_x, length, sigma, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomoz, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss(amp_x_hd, length_hd, sigma_hd, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

In [ ]:
ex_amplitude=0.25
lengths = np.linspace(0e-9, 200e-9, 51)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi x-axis pulse length'), filename='Rabi', output=False)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(measurement ['X'][2], measurement ['Y'][2], measurement ['Z'][2], c='r', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim([-0.6, 0.6])
ax.set_ylim([-0.6, 0.6])
ax.set_zlim([-0.6, 0.6])

In [ ]:
ex_amplitude = 0.25
pause_length = 16e-9
lengths = np.linspace(0e-9, 2000e-9, 2001)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(1j*ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi y-axis pulse length'), filename='Rabi', output=False)

### Optimizing phase between x and y pulses

In [ ]:
c_phase = 0
c_n = 0

adc.set_software_averages(1)
adc.set_software_nums_multi(3)

def set_n(n):
    global c_n
    global c_phase
    c_n = n 
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

def set_y_phase(phase):
    global c_n
    global c_phase
    c_phase = phase
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

phases = np.linspace(np.pi/2.*0.8, np.pi/2.*1.2, 11, endpoint=False)
num_pulses = np.arange(0,5)
measurement = sweep.sweep(tomo, (phases, set_y_phase, 'y-pulse phase'), (num_pulses, set_n, 'y pulse num'), filename='y-phase', output=False)

In [ ]:
phases[7]

In [ ]:
plt.plot(np.asarray(results).T)

In [ ]:
adc.stop()

In [ ]:
plt.close()

# load calibration data

In [ ]:
import pickle
f0 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-46\\Voltage ro calibration.pkl', 'rb')
f1 = open('D:\\qtlab\\ReiData\\data\\2017-04-26\\13-11-14\\Voltage ro calibration.pkl', 'rb')
cal0 = pickle.load(f0)
cal1 = pickle.load(f1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt4
cal0_nomean = cal0[1]['Voltage'][2][0,:,:]-np.mean(cal0[1]['Voltage'][2][0,:,:])
cal1_nomean = cal0[1]['Voltage'][2][1,:,:]-np.mean(cal0[1]['Voltage'][2][1,:,:])

cal0_mean = np.mean(cal0_nomean, axis=0)
cal1_mean = np.mean(cal1_nomean, axis=0)

In [ ]:
cal_mean = (cal0_mean+cal1_mean)/2.
cal_diff = (cal1_mean-cal0_mean)

plt.figure('Samples')
plt.plot(np.mean(cal0_nomean, axis=0), label='|0>')
plt.plot(np.mean(cal1_nomean, axis=0), label='|1>')
plt.legend()

plt.figure('Offset & feature')
plt.plot(cal_mean, label='mean')
plt.plot(cal_diff, label='diff')
plt.legend()

In [ ]:
plt.figure('Fourier domain')
plt.plot(np.abs(np.fft.fft(cal0_mean)), label='|0>')
plt.plot(np.abs(np.fft.fft(cal1_mean)), label='|1>') 
plt.plot(np.abs(np.fft.fft(cal_mean)), label='mean')
plt.plot(np.abs(np.fft.fft(cal_diff)), label='diff')
plt.legend()

feature = np.conj(cal_diff/np.sum(np.abs(cal_diff**2)))

In [ ]:
coeffs0 = np.dot(cal0_nomean-cal_mean, feature)
coeffs1 = np.dot(cal1_nomean-cal_mean, feature)
predictions = np.asarray([np.real(coeffs0), np.real(coeffs1)])

In [ ]:
hist_all, bins = np.histogram(predictions, bins='auto')
proba_points = (bins[1:]+bins[:-1])/2.
hists = []
for y in range(2):
    hists.append(np.histogram(predictions[y,:], bins=bins)[0])

hists = np.asarray(hists, dtype=float)
probabilities = hists/hist_all
naive_probabilities = np.asarray([proba_points<0, proba_points>0], dtype=float)
probabilities[np.isnan(probabilities)] = naive_probabilities[np.isnan(probabilities)]
predictor = lambda x: np.interp(x, proba_points, probabilities[1,:], left=0., right=1.)
calib_proba_points = proba_points
calib_proba = probabilities[1,:]
calib_hists = hists

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
roc = roc_curve([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))
roc_auc = roc_auc_score([0]*coeffs0.size+[1]*coeffs0.size, predictor(predictions.ravel()))

In [ ]:
plt.figure('ROC')
plt.plot(roc[0], roc[1])
plt.xlabel('True Positive Rate')
plt.ylabel('True Negative Rate')
roc_auc

In [ ]:
plt.figure('probability curve')
plt.plot(calib_proba_points, calib_proba)

In [ ]:
plt.figure('Readout hists')
plt.bar(proba_points-0.25, hists[0], width=0.05, label='|0>')
plt.bar(proba_points+0.25, hists[1], width=0.05, label='|1>')
plt.legend()

In [ ]:
plt.close()

In [ ]:
cal0_nomean.shape

In [ ]:
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal0_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')
plt.figure(figsize=(12,3))
plt.imshow(np.sum(np.real(np.reshape(cal1_nomean, (200, 100, 1024))), axis=0), cmap='RdBu')

In [ ]:
plt.plot(np.mean(cal0_nomean, axis=0))
plt.plot(np.mean(cal1_nomean, axis=0))